In [1]:
import pandas as pd
import seaborn as sns
import re
from pandasql import sqldf
import statistics
from tqdm import tqdm
import pgeocode
import pickle
pysqldf = lambda q: sqldf(q, globals())
from sklearn.linear_model import LinearRegression
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
prop_list = pd.read_csv("../data/raw/listing_properties.csv")

In [3]:
prop_list

,Unnamed: 0,beds,baths,parking,title,price,address,promoType,listingType,propertyType,status,mode,isRural,houses,bond,id,availableFrom,landArea,buildingArea,method
0,0,1,1,0,$330,$330,"409/757 Bourke, Docklands VIC 3008",basic,rent,Apartment / Unit / Flat,recentlyUpdated,rent,False,0,1433.0,14303265,NaN,NaN,NaN,NaN
1,1,2,1,1,$430 pw,$430 pw,"38/60-68 Gladesville Boulevard, Patterson Lake...",branded,rent,Apartment / Unit / Flat,new,rent,False,0,1868.0,16089992,"Monday, 28th June 2021",NaN,NaN,NaN
2,2,4,3,2,$585,$585,"1 Liverpool Circuit, Craigieburn VIC 3064",branded,rent,House,recentlyUpdated,rent,False,0,NaN,15279569,"Friday, 25th November 2022",NaN,NaN,NaN
3,3,3,1,1,$420,$420,"8 Stoneman Turn, Caroline Springs VIC 3023",branded,rent,House,new,rent,False,0,1820.0,16089977,"Monday, 26th September 2022",NaN,NaN,NaN
4,4,1,1,1,$640 per week,$640 per week,"1510/1 Freshwater Place, Southbank VIC 3006",platinum,rent,Apartment / Unit / Flat,new,rent,False,0,2781.0,16089972,"Monday, 22nd August 2022",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13748,13748,4,2,2,$400 Per Week pw,$400 Per Week pw,"7 Ebony Way, Tarneit VIC 3029",basic,rent,House,live,rent,False,0,1738.0,15434092,"Wednesday, 27th October 2021",NaN,NaN,NaN
13749,13749,1,1,0,$435 per week,$435 per week,"17/416 St Kilda Rd, Melbourne VIC 3004",basic,rent,Apartment / Unit / Flat,live,rent,False,0,2500.0,14409092,"Saturday, 9th October 2021",NaN,50.0,NaN
13750,13750,3,2,4,$350 Per Week pw,$350 Per Week pw,"16 Goulburn Court, Werribee VIC 3030",basic,rent,House,live,rent,False,0,1521.0,15430756,"Wednesday, 6th October 2021",NaN,NaN,NaN
13751,13751,4,2,1,$600,$600,"18 Jessie Street, Preston VIC 3072",basic,rent,House,live,rent,False,0,2600.0,15428527,"Tuesday, 5th October 2021",NaN,NaN,NaN


In [4]:
prop_list[prop_list['landArea'].notna()]

,Unnamed: 0,beds,baths,parking,title,price,address,promoType,listingType,propertyType,status,mode,isRural,houses,bond,id,availableFrom,landArea,buildingArea,method
11,11,4,2,2,$630 weekly,$630 weekly,"90 Shepherd Rd, Glen Waverley VIC 3150",basic,rent,House,new,rent,False,0,2738.0,16089938,"Friday, 23rd September 2022",652.0,175.0,NaN
13,13,2,1,0,$700 per week,$700 per week,"15 Dally Street, Clifton Hill VIC 3068",basic,rent,House,recentlyUpdated,rent,False,0,3042.0,15478015,"Friday, 14th October 2022",158.0,95.0,NaN
14,14,4,2,2,$500,$500,"20 Knightsford Avenue, Clyde VIC 3978",branded,rent,House,new,rent,False,0,2173.0,16089930,"Wednesday, 12th October 2022",375.0,NaN,NaN
18,18,3,1,2,$360 Per Week,$360 Per Week,"5 Roxburgh Road, Yallourn North VIC 3825",branded,rent,House,new,rent,False,0,1560.0,16089911,"Wednesday, 5th October 2022",634.0,NaN,NaN
35,35,3,2,2,$460 pw,$460 pw,"2/17 Nelson Road, Trafalgar VIC 3824",basic,rent,Townhouse,new,rent,False,0,1999.0,16089810,"Sunday, 25th September 2022",317.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13667,13667,3,2,11,$1180 PER WEEK $5113 PER MONTH,$1180 PER WEEK $5113 PER MONTH,"2531 STEIGLITZ ROAD, Steiglitz VIC 3331",branded,rent,Acreage / Semi-Rural,live,rent,True,0,5000.0,15714546,"Sunday, 20th February 2022",600000.0,NaN,NaN
13670,13670,4,2,2,Contact Agent,Contact Agent,"13 Roaming Drive, Fraser Rise VIC 3336",branded,rent,House,live,rent,False,0,NaN,15711847,"Wednesday, 6th April 2022",352.0,NaN,NaN
13677,13677,2,2,1,UNDER APPLICATION,UNDER APPLICATION,"2/50 Fraser Crescent, Ocean Grove VIC 3226",branded,rent,Townhouse,live,rent,False,0,2086.0,15649364,"Saturday, 5th February 2022",197.0,NaN,NaN
13732,13732,4,2,2,$410.00 Per Week pw,$410.00 Per Week pw,"10 Clearwood Drive, Truganina VIC 3029",basic,rent,House,live,rent,False,0,1782.0,15454576,"Friday, 10th December 2021",360.0,22.0,NaN


In [4]:
prop_subset = prop_list[['id','price','beds','baths','parking','title','address','propertyType','status','bond']]

In [5]:
prop_subset.isnull().sum()

id                0
price             0
beds              0
baths             0
parking           0
title             0
address           0
propertyType      0
status            0
bond            890
dtype: int64

In [6]:
prop_subset_clean = prop_subset.dropna()

In [7]:
prop_subset_clean.isnull().sum()

id              0
price           0
beds            0
baths           0
parking         0
title           0
address         0
propertyType    0
status          0
bond            0
dtype: int64

In [8]:
prop_subset_clean

,id,price,beds,baths,parking,title,address,propertyType,status,bond
0,14303265,$330,1,1,0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0
1,16089992,$430 pw,2,1,1,$430 pw,"38/60-68 Gladesville Boulevard, Patterson Lake...",Apartment / Unit / Flat,new,1868.0
3,16089977,$420,3,1,1,$420,"8 Stoneman Turn, Caroline Springs VIC 3023",House,new,1820.0
4,16089972,$640 per week,1,1,1,$640 per week,"1510/1 Freshwater Place, Southbank VIC 3006",Apartment / Unit / Flat,new,2781.0
6,16089969,$550 PER WEEK,4,2,2,$550 PER WEEK,"6 Plenty Drive, Warragul VIC 3820",House,new,2390.0
...,...,...,...,...,...,...,...,...,...,...
13748,15434092,$400 Per Week pw,4,2,2,$400 Per Week pw,"7 Ebony Way, Tarneit VIC 3029",House,live,1738.0
13749,14409092,$435 per week,1,1,0,$435 per week,"17/416 St Kilda Rd, Melbourne VIC 3004",Apartment / Unit / Flat,live,2500.0
13750,15430756,$350 Per Week pw,3,2,4,$350 Per Week pw,"16 Goulburn Court, Werribee VIC 3030",House,live,1521.0
13751,15428527,$600,4,2,1,$600,"18 Jessie Street, Preston VIC 3072",House,live,2600.0


In [10]:
def get_price(e):
    s = re.search('\$[0-9]*\,?\d*\.?\d*', e)
    if (s == None):
        return None
    else:
        final = s.group().strip('$')
        final = final.replace(',','')
        return final

In [11]:
prop_subset_clean['price'] = prop_subset_clean['price'].apply(get_price)

/tmp/ipykernel_392/2730741709.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prop_subset_clean['price'] = prop_subset_clean['price'].apply(get_price)


In [12]:
prop_subset_clean

,id,price,beds,baths,parking,title,address,propertyType,status,bond
0,14303265,330,1,1,0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0
1,16089992,430,2,1,1,$430 pw,"38/60-68 Gladesville Boulevard, Patterson Lake...",Apartment / Unit / Flat,new,1868.0
3,16089977,420,3,1,1,$420,"8 Stoneman Turn, Caroline Springs VIC 3023",House,new,1820.0
4,16089972,640,1,1,1,$640 per week,"1510/1 Freshwater Place, Southbank VIC 3006",Apartment / Unit / Flat,new,2781.0
6,16089969,550,4,2,2,$550 PER WEEK,"6 Plenty Drive, Warragul VIC 3820",House,new,2390.0
...,...,...,...,...,...,...,...,...,...,...
13748,15434092,400,4,2,2,$400 Per Week pw,"7 Ebony Way, Tarneit VIC 3029",House,live,1738.0
13749,14409092,435,1,1,0,$435 per week,"17/416 St Kilda Rd, Melbourne VIC 3004",Apartment / Unit / Flat,live,2500.0
13750,15430756,350,3,2,4,$350 Per Week pw,"16 Goulburn Court, Werribee VIC 3030",House,live,1521.0
13751,15428527,600,4,2,1,$600,"18 Jessie Street, Preston VIC 3072",House,live,2600.0


In [13]:
prop_subset_clean['price'] = pd.to_numeric(prop_subset_clean['price'])

/tmp/ipykernel_392/2753503525.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prop_subset_clean['price'] = pd.to_numeric(prop_subset_clean['price'])


In [14]:
prop_subset_clean

,id,price,beds,baths,parking,title,address,propertyType,status,bond
0,14303265,330.0,1,1,0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0
1,16089992,430.0,2,1,1,$430 pw,"38/60-68 Gladesville Boulevard, Patterson Lake...",Apartment / Unit / Flat,new,1868.0
3,16089977,420.0,3,1,1,$420,"8 Stoneman Turn, Caroline Springs VIC 3023",House,new,1820.0
4,16089972,640.0,1,1,1,$640 per week,"1510/1 Freshwater Place, Southbank VIC 3006",Apartment / Unit / Flat,new,2781.0
6,16089969,550.0,4,2,2,$550 PER WEEK,"6 Plenty Drive, Warragul VIC 3820",House,new,2390.0
...,...,...,...,...,...,...,...,...,...,...
13748,15434092,400.0,4,2,2,$400 Per Week pw,"7 Ebony Way, Tarneit VIC 3029",House,live,1738.0
13749,14409092,435.0,1,1,0,$435 per week,"17/416 St Kilda Rd, Melbourne VIC 3004",Apartment / Unit / Flat,live,2500.0
13750,15430756,350.0,3,2,4,$350 Per Week pw,"16 Goulburn Court, Werribee VIC 3030",House,live,1521.0
13751,15428527,600.0,4,2,1,$600,"18 Jessie Street, Preston VIC 3072",House,live,2600.0


In [15]:
prop_subset_clean = prop_subset_clean.dropna(axis=0)

In [21]:
prop_subset_clean = prop_subset_clean.reset_index()

In [22]:
prop_subset_clean['price'].sort_values(ascending = True)

7862        40.0
10950       40.0
12075       50.0
4277        50.0
12350       50.0
          ...   
6364      4000.0
11403     5000.0
7512      5500.0
10550    39000.0
11644    41250.0
Name: price, Length: 12487, dtype: float64

In [23]:
prop_subset_clean.iloc[11644,:]

index                                                       12751
id                                                       15994278
price                                                     41250.0
beds                                                            0
baths                                                           0
parking                                                         0
title                                                   $41250 pa
address         Level 1, 301/13-15 Lake Street, Caroline Sprin...
propertyType                              Apartment / Unit / Flat
status                                                       live
bond                                                       9519.0
Name: 11644, dtype: object

In [26]:
prop_subset_clean = prop_subset_clean[prop_subset_clean['beds'] > 0]

In [29]:
prop_subset_clean.pop('index')

0            0
1            1
2            3
3            4
4            6
         ...  
12482    13748
12483    13749
12484    13750
12485    13751
12486    13752
Name: index, Length: 12405, dtype: int64

In [32]:
prop_subset_clean.to_csv('../data/curated/cleaned_property_listing_data.csv')

### Now we clean all existing scraped dataset

In [13]:
prop = pd.read_csv('../data/curated/cleaned_property_listing_data.csv')
schools = pd.read_csv('../data/raw/closest_school.csv')
suburb_stat = pd.read_csv('../data/raw/suburb_stat.csv')
neigh_stat = pd.read_csv('../data/raw/neigh_stat.csv')

In [3]:
prop['bed'] = pd.to_numeric(prop['beds'])
prop['baths'] = pd.to_numeric(prop['baths'])
prop['parking'] = pd.to_numeric(prop['parking'])
prop['bond'] = pd.to_numeric(prop['bond'])

In [14]:
suburb_stat

,suburb,medianRentPrice,population,avgAge,owners,renters,families,singles,id
0,Docklands,380,10962,20 to 39,0.356243,0.643757,0.393689,0.606311,14303265
1,Patterson Lakes,430,7565,40 to 59,0.835075,0.164925,0.527570,0.472430,16089992
2,Craigieburn,450,50336,20 to 39,0.716903,0.283097,0.556267,0.443733,15279569
3,Caroline Springs,410,24200,20 to 39,0.766538,0.233462,0.571566,0.428434,16089977
4,Southbank,360,18706,20 to 39,0.347128,0.652872,0.312419,0.687581,16089972
...,...,...,...,...,...,...,...,...,...
13711,Tarneit,400,34556,20 to 39,0.699615,0.300385,0.611806,0.388194,15434092
13712,Melbourne,375,0,NaN,0.000000,0.000000,0.000000,0.000000,14409092
13713,Werribee,340,40348,20 to 39,0.679556,0.320444,0.463697,0.536303,15430756
13714,Preston,600,32856,20 to 39,0.588015,0.411985,0.399137,0.600863,15428527


In [21]:
prop.pop('bed')
prop.pop('Unnamed: 0')
schools.pop('Unnamed: 0')
suburb_stat.pop('Unnamed: 0')
neigh_stat.pop('Unnamed: 0')

0            0
1            1
2            2
3            3
4            4
         ...  
12400    12400
12401    12401
12402    12402
12403    12403
12404    12404
Name: Unnamed: 0, Length: 12405, dtype: int64

In [22]:
prop.to_csv('../data/curated/cleaned_property_listing_data.csv', index = False)

In [12]:
schools.to_csv('../data/raw/closest_school.csv', index = False)
suburb_stat.to_csv('../data/raw/suburb_stat.csv', index = False)
neigh_stat.to_csv('../data/raw/neigh_stat.csv', index = False)

In [15]:
neigh_stat

,age0To19,age20To39,age40To59,age60Plus,longTermResident,owner,renter,family,single,id
0,0.129199,0.642119,0.195090,0.033592,0.136929,0.295322,0.704678,0.512945,0.487055,14303265
1,0.210918,0.248139,0.332506,0.208437,0.478632,0.639535,0.360465,0.576052,0.423948,16089992
2,0.328173,0.229102,0.346749,0.095975,0.000000,0.800000,0.200000,0.647887,0.352113,15279569
3,0.331512,0.251366,0.320583,0.096539,0.604072,0.777778,0.222222,0.600000,0.400000,16089977
4,0.093800,0.356121,0.356121,0.193959,0.202410,0.537366,0.462633,0.568019,0.431981,16089972
...,...,...,...,...,...,...,...,...,...,...
13711,0.362153,0.427406,0.205546,0.004894,0.000000,0.374233,0.625767,0.602015,0.397985,15434092
13712,0.113907,0.540397,0.255629,0.090066,0.240545,0.355030,0.644970,0.419152,0.580848,14409092
13713,0.233898,0.277966,0.315254,0.172881,0.642857,0.663366,0.336634,0.570732,0.429268,15430756
13714,0.208772,0.356140,0.231579,0.203509,0.662698,0.641304,0.358696,0.493088,0.506912,15428527


In [21]:
neigh_stat

,age0To19,age20To39,age40To59,age60Plus,longTermResident,owner,renter,family,single,id
0,0.129199,0.642119,0.195090,0.033592,0.136929,0.295322,0.704678,0.512945,0.487055,14303265
1,0.210918,0.248139,0.332506,0.208437,0.478632,0.639535,0.360465,0.576052,0.423948,16089992
2,0.328173,0.229102,0.346749,0.095975,0.000000,0.800000,0.200000,0.647887,0.352113,15279569
3,0.331512,0.251366,0.320583,0.096539,0.604072,0.777778,0.222222,0.600000,0.400000,16089977
4,0.093800,0.356121,0.356121,0.193959,0.202410,0.537366,0.462633,0.568019,0.431981,16089972
...,...,...,...,...,...,...,...,...,...,...
13711,0.362153,0.427406,0.205546,0.004894,0.000000,0.374233,0.625767,0.602015,0.397985,15434092
13712,0.113907,0.540397,0.255629,0.090066,0.240545,0.355030,0.644970,0.419152,0.580848,14409092
13713,0.233898,0.277966,0.315254,0.172881,0.642857,0.663366,0.336634,0.570732,0.429268,15430756
13714,0.208772,0.356140,0.231579,0.203509,0.662698,0.641304,0.358696,0.493088,0.506912,15428527


In [16]:
prop.rename(columns={'id':'listing_id'}, inplace = True)
suburb_stat.rename(columns={'id':'listing_id'}, inplace = True)
neigh_stat.rename(columns={'id':'listing_id'}, inplace = True)

In [17]:
neigh_stat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13716 entries, 0 to 13715
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age0To19          13716 non-null  float64
 1   age20To39         13716 non-null  float64
 2   age40To59         13716 non-null  float64
 3   age60Plus         13716 non-null  float64
 4   longTermResident  13716 non-null  float64
 5   owner             13716 non-null  float64
 6   renter            13716 non-null  float64
 7   family            13716 non-null  float64
 8   single            13716 non-null  float64
 9   listing_id        13716 non-null  int64  
dtypes: float64(9), int64(1)
memory usage: 1.0 MB


In [37]:
suburb_stat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13716 entries, 0 to 13715
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   suburb           13716 non-null  object 
 1   medianRentPrice  13716 non-null  int64  
 2   population       13716 non-null  int64  
 3   avgAge           12892 non-null  object 
 4   owners           13716 non-null  float64
 5   renters          13716 non-null  float64
 6   families         13716 non-null  float64
 7   singles          13716 non-null  float64
 8   listing_id       13716 non-null  int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 964.5+ KB


In [18]:
schools.rename(columns={'name':"school_name", 'address':'school_address'})

,id,educationLevel,school_name,distance,state,postCode,year,gender,type,school_address,url,listing_id
0,NaN,combined,Eltham College - King Street Campus,708.921822,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN,14303265
1,NaN,secondary,Hester Hornbrook Academy - City Campus,802.849893,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN,14303265
2,NaN,secondary,Ozford College - Ozford College Campus,899.683164,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN,14303265
3,NaN,secondary,Ozford College,900.734191,VIC,3000,11-12,CoEd,Private,"Melbourne, VIC 3000",http://www.ozford.edu.au,14303265
4,NaN,primary,Docklands Primary School,916.135369,VIC,3008,Prep-6,CoEd,Government,"Docklands, VIC 3008",NaN,14303265
...,...,...,...,...,...,...,...,...,...,...,...,...
167001,1999.0,secondary,Simonds Catholic College,942.720484,VIC,3003,7-12,Boys,Catholic,"West Melbourne, VIC 3003",http://www.sccmelb.catholic.edu.au,15384018
167002,NaN,secondary,St Joseph's Flexible Learning Centre,969.075211,VIC,3051,7-12,CoEd,Catholic,"North Melbourne, VIC 3051",http://www.youthplus.edu.au,15384018
167003,NaN,combined,Eltham College - King Street Campus,1012.666859,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN,15384018
167004,1266.0,primary,Carlton Gardens Primary School,1058.486337,VIC,3053,Prep-6,CoEd,Government,"Carlton, VIC 3053",http://www.carltongardens.vic.edu.au,15384018


In [23]:
prop

,listing_id,price,beds,baths,parking,title,address,propertyType,status,bond,postcode,suburb
0,14303265,330.0,1,1,0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0,3008,Docklands
1,16089992,430.0,2,1,1,$430 pw,"38/60-68 Gladesville Boulevard, Patterson Lake...",Apartment / Unit / Flat,new,1868.0,3197,Patterson Lakes
2,16089977,420.0,3,1,1,$420,"8 Stoneman Turn, Caroline Springs VIC 3023",House,new,1820.0,3023,Caroline Springs
3,16089972,640.0,1,1,1,$640 per week,"1510/1 Freshwater Place, Southbank VIC 3006",Apartment / Unit / Flat,new,2781.0,3006,Southbank
4,16089969,550.0,4,2,2,$550 PER WEEK,"6 Plenty Drive, Warragul VIC 3820",House,new,2390.0,3820,Warragul
...,...,...,...,...,...,...,...,...,...,...,...,...
12400,15434092,400.0,4,2,2,$400 Per Week pw,"7 Ebony Way, Tarneit VIC 3029",House,live,1738.0,3029,Tarneit
12401,14409092,435.0,1,1,0,$435 per week,"17/416 St Kilda Rd, Melbourne VIC 3004",Apartment / Unit / Flat,live,2500.0,3004,Melbourne
12402,15430756,350.0,3,2,4,$350 Per Week pw,"16 Goulburn Court, Werribee VIC 3030",House,live,1521.0,3030,Werribee
12403,15428527,600.0,4,2,1,$600,"18 Jessie Street, Preston VIC 3072",House,live,2600.0,3072,Preston


In [24]:
prop['address']
suburbs = []
postcode = []
for a in prop['address']:
    temp = a.split("VIC")
    sub = temp[0].split(",")[-1]
#     print(sub)
    suburbs.append(sub.strip())
    postcode.append(int(temp[1]))
    
    
prop['suburb'] = suburbs
prop['postcode'] = postcode

In [60]:
prop.to_csv('../data/curated/cleaned_property_listing_data.csv')

### Getting Aggregated data by type of rental property and suburb

In [1]:
%cd ../scripts
from get_agg import get_agg
agg = get_agg(write=True) # sdf is saved to a location
%cd ../notebooks

/mnt/c/Users/User/OneDrive/Desktop/Uni - 2022/sem2/Applied Data Science/Proj2/generic-real-estate-consulting-project-group-25/scripts


100%|███████████████████████████████████████████████████████████████████████████████| 2935/2935 [04:50<00:00, 10.10it/s]

/mnt/c/Users/User/OneDrive/Desktop/Uni - 2022/sem2/Applied Data Science/Proj2/generic-real-estate-consulting-project-group-25/notebooks


In [29]:
agg = pd.read_csv('../data/curated/2022_aggregated/agg_bed_avgPrice_sub.csv')
agg

,beds,avg_price,count,propertyType,suburb,median
0,1,500.0,1,Acreage / Semi-Rural,Oaklands Junction,500.0
1,1,401.0,15,Apartment / Unit / Flat,Abbotsford,410.0
2,1,290.0,1,Apartment / Unit / Flat,Airport West,290.0
3,1,417.5,2,Apartment / Unit / Flat,Albert Park,417.5
4,1,320.0,2,Apartment / Unit / Flat,Alphington,320.0
...,...,...,...,...,...,...
2930,7,1250.0,1,House,Brunswick,1250.0
2931,7,170.0,1,House,Dandenong,170.0
2932,8,1950.0,1,House,Balaclava,1950.0
2933,8,1300.0,1,House,Mordialloc,1300.0


In [2]:
agg

,beds,avg_price,count,propertyType,suburb,median
0,1,500.0,1,Acreage / Semi-Rural,Oaklands Junction,500.0
1,1,401.0,15,Apartment / Unit / Flat,Abbotsford,410.0
2,1,290.0,1,Apartment / Unit / Flat,Airport West,290.0
3,1,417.5,2,Apartment / Unit / Flat,Albert Park,417.5
4,1,320.0,2,Apartment / Unit / Flat,Alphington,320.0
...,...,...,...,...,...,...
2930,7,1250.0,1,House,Brunswick,1250.0
2931,7,170.0,1,House,Dandenong,170.0
2932,8,1950.0,1,House,Balaclava,1950.0
2933,8,1300.0,1,House,Mordialloc,1300.0


### Checking Historical Data

In [78]:
bed_1_flat = pd.read_csv('../data/raw/Rental_Price_historical/1_bed_flat.csv')

In [79]:
bed_1_flat

,Unnamed: 0,Unnamed: 1,Mar-00,Unnamed: 3,Jun-00,Unnamed: 5,Sep-00,Unnamed: 7,Dec-00,Unnamed: 9,...,Mar-20,Unnamed: 163,Jun-20,Unnamed: 165,Sep-20,Unnamed: 167,Dec-20,Unnamed: 169,Mar-21,Unnamed: 171
0,NaN,NaN,Count,Median,Count,Median,Count,Median,Count,Median,...,Count,Median,Count,Median,Count,Median,Count,Median,Count,Median
1,Inner Melbourne,Albert Park-Middle Park-West St Kilda,352,$165,346,$165,376,$170,367,$175,...,230,$370,228,$355,231,$350,243,$350,263,$340
2,NaN,Armadale,210,$150,210,$150,211,$155,211,$160,...,205,$375,208,$368,170,$350,184,$350,206,$350
3,NaN,Carlton North,86,$150,77,$155,74,$150,65,$150,...,70,$363,67,$360,72,$350,64,$350,68,$338
4,NaN,Carlton-Parkville,297,$170,297,$170,310,$175,343,$180,...,"3,354",$360,"3,239",$356,"2,919",$350,"2,829",$340,"2,097",$300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,NaN,Wanagaratta,51,$85,45,$85,43,$85,46,$85,...,69,$195,67,$195,61,$190,56,$195,52,$195
156,NaN,Warragul,13,$80,11,$75,12,$90,10,$90,...,-,-,-,-,-,-,-,-,-,-
157,NaN,Warrnambool,113,$75,103,$75,107,$75,103,$80,...,61,$200,65,$220,63,$220,63,$230,55,$250
158,NaN,Wodonga,76,$85,72,$85,77,$85,83,$85,...,46,$180,57,$185,56,$185,56,$185,59,$200


In [31]:
pop = pd.read_csv('../data/raw/Population_by_suburb.csv')
age_data = pd.read_csv('../data/raw/2021Census_G01_VIC_POA.csv')
monetary_data = pd.read_csv('../data/raw/2021Census_G02_VIC_POA.csv')

### Impuation of missing data using regression for suburb level age and socio-economic data

In [2]:
# Australian CPI from https://www.ato.gov.au/rates/consumer-price-index/
CPI_2021 = (117.9 + 118.8 + 119.7 + 121.3)/4
CPI_2016 = (108.2 + 108.6 + 109.4 + 110)/4
CPI_2011 = (98.3 + 99.2 + 99.8 + 99.8)/4

In [3]:
CPI_2021, CPI_2016, CPI_2011

(119.425, 109.05000000000001, 99.275)

In [37]:
age_data2011 = pd.read_csv('../data/raw/Other_sub_data/2011/2011Census_B01_VIC_POA_short.csv')
monetary_data2011 = pd.read_csv('../data/raw/Other_sub_data/2011/2011Census_B02_VIC_POA_short.csv')
age_data2016 = pd.read_csv('../data/raw/Other_sub_data/2016/2016Census_G01_VIC_POA.csv')
monetary_data2016 = pd.read_csv('../data/raw/Other_sub_data/2016/2016Census_G02_VIC_POA.csv')
age_data2021 = pd.read_csv('../data/raw/Other_sub_data/2021/2021Census_G01_VIC_POA.csv')
monetary_data2021 = pd.read_csv('../data/raw/Other_sub_data/2021/2021Census_G02_VIC_POA.csv')


In [38]:
monetary_data2011.rename(columns = {'region_id': 'Postcode'}, inplace = True)
monetary_data2016.rename(columns = {'POA_CODE_2016': 'Postcode'}, inplace = True)
monetary_data2021.rename(columns = {'POA_CODE_2021': 'Postcode'}, inplace = True)

In [39]:
monetary_data2011

,Postcode,Median_age_persons,Median_mortgage_repay_monthly,Median_Tot_prsnl_inc_weekly,Median_rent_weekly_,Median_Tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_Tot_hhd_inc_weekly,Average_household_size
0,POA3000,26,1950,545,402,1446,1.3,994,1.8
1,POA3002,36,2383,1164,415,2693,1.1,1989,1.8
2,POA3003,30,2400,800,441,2006,1.2,1704,2.2
3,POA3004,34,2167,1103,450,2544,1.2,1921,1.9
4,POA3005,30,2167,1056,500,2416,1.2,1944,2.0
...,...,...,...,...,...,...,...,...,...
662,POA3988,40,1343,521,165,1214,1.1,1032,2.7
663,POA3991,47,1083,461,160,1062,1.1,877,2.5
664,POA3992,39,1292,525,195,1174,1.1,1014,2.4
665,POA3995,44,1300,445,220,1064,1.1,814,2.3


In [7]:
monetary_data2016

,Postcode,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size
0,POA3000,26,1700,431,451,1422,1.2,955,2.0
1,POA3002,38,2192,1341,451,3120,1.0,2285,1.9
2,POA3003,30,2006,852,450,2246,1.1,1766,2.2
3,POA3004,35,2000,1187,473,2526,1.0,2006,1.8
4,POA3005,33,2184,1139,584,2562,1.0,2182,2.1
...,...,...,...,...,...,...,...,...,...
693,POA3990,51,1467,641,275,1312,0.6,1075,2.2
694,POA3991,50,1355,428,200,1009,0.7,811,2.4
695,POA3992,42,1419,543,225,1256,0.7,1052,2.3
696,POA3995,48,1300,480,230,1129,0.7,840,2.2


In [8]:
monetary_data2021

,Postcode,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size
0,POA3000,28,1733,768,370,1857,1.1,1306,1.7
1,POA3002,42,2383,1532,480,3733,0.9,2345,1.8
2,POA3003,31,2106,1058,388,2406,1.1,1788,1.9
3,POA3004,37,2000,1343,451,2727,1.0,2090,1.7
4,POA3006,31,1900,1171,411,2450,1.0,1927,1.8
...,...,...,...,...,...,...,...,...,...
689,POA3990,56,2134,621,308,1687,0.8,1562,2.5
690,POA3991,49,1500,550,225,1674,0.7,1262,2.4
691,POA3992,43,1517,699,280,1601,0.7,1402,2.4
692,POA3995,50,1300,569,280,1392,0.7,1034,2.2


#### Converting Postcode to suburb

In [40]:
nomi = pgeocode.Nominatim('au')
info = nomi.query_postal_code("3002")

In [12]:
info['place_name']

'East Melbourne'

In [66]:
poa_dict = {}
for p in monetary_data2021['POA_CODE_2021']:
    poa = p.split('POA')[1]
    info_p = nomi.query_postal_code(poa)
    poa_dict[poa] = info_p['place_name']

In [65]:
outfile = open('../data/curated/postcode_dict','wb')
pickle.dump(poa_dict,outfile)

In [63]:
poa_dict

{'3000': 'Melbourne',
 '3002': 'East Melbourne',
 '3003': 'West Melbourne',
 '3004': 'St Kilda Road Melbourne, St Kilda Road Central, Melbourne',
 '3006': 'South Wharf, Southbank',
 '3008': 'Docklands',
 '3011': 'Seddon, Seddon West, Footscray',
 '3012': 'Brooklyn, Kingsville, Maidstone, Tottenham, West Footscray',
 '3013': 'Yarraville West, Yarraville',
 '3015': 'Spotswood, South Kingsville, Newport',
 '3016': 'Williamstown North, Williamstown',
 '3018': 'Seaholme, Altona',
 '3019': 'Braybrook, Robinson',
 '3020': 'Sunshine West, Sunshine, Sunshine North, Glengala, Albion',
 '3021': 'St Albans, Kings Park, Kealba, Albanvale',
 '3022': 'Deer Park East, Ardeer',
 '3023': 'Cairnlea, Caroline Springs, Burnside, Burnside Heights, Ravenhall, Deer Park North, Deer Park',
 '3024': 'Mount Cottrell, Mambourin, Wyndham Vale',
 '3025': 'Altona East, Altona Gate, Altona North',
 '3026': 'Laverton North',
 '3027': 'Williams Landing',
 '3028': 'Laverton, Altona Meadows, Seabrook',
 '3029': 'Truganin

In [97]:
monetary_data2011

,region_id,Median_age_persons,Median_mortgage_repay_monthly,Median_Tot_prsnl_inc_weekly,Median_rent_weekly_,Median_Tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_Tot_hhd_inc_weekly,Average_household_size
0,POA3000,26,1950,545,402,1446,1.3,994,1.8
1,POA3002,36,2383,1164,415,2693,1.1,1989,1.8
2,POA3003,30,2400,800,441,2006,1.2,1704,2.2
3,POA3004,34,2167,1103,450,2544,1.2,1921,1.9
4,POA3005,30,2167,1056,500,2416,1.2,1944,2.0
...,...,...,...,...,...,...,...,...,...
662,POA3988,40,1343,521,165,1214,1.1,1032,2.7
663,POA3991,47,1083,461,160,1062,1.1,877,2.5
664,POA3992,39,1292,525,195,1174,1.1,1014,2.4
665,POA3995,44,1300,445,220,1064,1.1,814,2.3


In [9]:
def get_suburb_name(df):
    poa_name = []
    for p in df['Postcode']:
        poa = p.split('POA')[1]
        info_p = nomi.query_postal_code(poa)
        poa_name.append(info_p['place_name'])
    return poa_name

In [41]:
monetary_data2021['Suburb'] = get_suburb_name(monetary_data2021)
monetary_data2016['Suburb'] = get_suburb_name(monetary_data2016)
monetary_data2011['Suburb'] = get_suburb_name(monetary_data2011)

In [42]:
monetary_data2021.pop('Median_tot_hhd_inc_weekly')
monetary_data2016.pop('Median_tot_hhd_inc_weekly')
monetary_data2011.pop('Median_Tot_hhd_inc_weekly')

0       994
1      1989
2      1704
3      1921
4      1944
       ... 
662    1032
663     877
664    1014
665     814
666     965
Name: Median_Tot_hhd_inc_weekly, Length: 667, dtype: int64

In [16]:
monetary_data2016.to_csv('../data/test.csv')

In [43]:
def rename_all(df, year):
    for c in df:
        if (c != 'Postcode' and c != 'Suburb'):
            df.rename(columns = {c:c+'_'+str(year)}, inplace = True)

In [44]:
rename_all(monetary_data2011, 2011)
rename_all(monetary_data2016, 2016)
rename_all(monetary_data2021, 2021)

In [45]:
monetary_data2021

,Postcode,Median_age_persons_2021,Median_mortgage_repay_monthly_2021,Median_tot_prsnl_inc_weekly_2021,Median_rent_weekly_2021,Median_tot_fam_inc_weekly_2021,Average_num_psns_per_bedroom_2021,Average_household_size_2021,Suburb
0,POA3000,28,1733,768,370,1857,1.1,1.7,Melbourne
1,POA3002,42,2383,1532,480,3733,0.9,1.8,East Melbourne
2,POA3003,31,2106,1058,388,2406,1.1,1.9,West Melbourne
3,POA3004,37,2000,1343,451,2727,1.0,1.7,"St Kilda Road Melbourne, St Kilda Road Central..."
4,POA3006,31,1900,1171,411,2450,1.0,1.8,"South Wharf, Southbank"
...,...,...,...,...,...,...,...,...,...
689,POA3990,56,2134,621,308,1687,0.8,2.5,Glen Forbes
690,POA3991,49,1500,550,225,1674,0.7,2.4,Bass
691,POA3992,43,1517,699,280,1601,0.7,2.4,"Blackwood Forest, Dalyston, Ryanston, West Creek"
692,POA3995,50,1300,569,280,1392,0.7,2.2,"Kilcunda, North Wonthaggi, Cape Paterson, Ande..."


In [46]:
monetary_data2016

,Postcode,Median_age_persons_2016,Median_mortgage_repay_monthly_2016,Median_tot_prsnl_inc_weekly_2016,Median_rent_weekly_2016,Median_tot_fam_inc_weekly_2016,Average_num_psns_per_bedroom_2016,Average_household_size_2016,Suburb
0,POA3000,26,1700,431,451,1422,1.2,2.0,Melbourne
1,POA3002,38,2192,1341,451,3120,1.0,1.9,East Melbourne
2,POA3003,30,2006,852,450,2246,1.1,2.2,West Melbourne
3,POA3004,35,2000,1187,473,2526,1.0,1.8,"St Kilda Road Melbourne, St Kilda Road Central..."
4,POA3005,33,2184,1139,584,2562,1.0,2.1,World Trade Centre
...,...,...,...,...,...,...,...,...,...
693,POA3990,51,1467,641,275,1312,0.6,2.2,Glen Forbes
694,POA3991,50,1355,428,200,1009,0.7,2.4,Bass
695,POA3992,42,1419,543,225,1256,0.7,2.3,"Blackwood Forest, Dalyston, Ryanston, West Creek"
696,POA3995,48,1300,480,230,1129,0.7,2.2,"Kilcunda, North Wonthaggi, Cape Paterson, Ande..."


In [48]:
monetary_data2021

,Postcode,Median_age_persons_2021,Median_mortgage_repay_monthly_2021,Median_tot_prsnl_inc_weekly_2021,Median_rent_weekly_2021,Median_tot_fam_inc_weekly_2021,Average_num_psns_per_bedroom_2021,Average_household_size_2021,Suburb
0,POA3000,28,1733,768,370,1857,1.1,1.7,Melbourne
1,POA3002,42,2383,1532,480,3733,0.9,1.8,East Melbourne
2,POA3003,31,2106,1058,388,2406,1.1,1.9,West Melbourne
3,POA3004,37,2000,1343,451,2727,1.0,1.7,"St Kilda Road Melbourne, St Kilda Road Central..."
4,POA3006,31,1900,1171,411,2450,1.0,1.8,"South Wharf, Southbank"
...,...,...,...,...,...,...,...,...,...
689,POA3990,56,2134,621,308,1687,0.8,2.5,Glen Forbes
690,POA3991,49,1500,550,225,1674,0.7,2.4,Bass
691,POA3992,43,1517,699,280,1601,0.7,2.4,"Blackwood Forest, Dalyston, Ryanston, West Creek"
692,POA3995,50,1300,569,280,1392,0.7,2.2,"Kilcunda, North Wonthaggi, Cape Paterson, Ande..."


In [49]:
def get_postcode(df):
    postcode = []
    for p in df.loc[:,'Postcode']:
        curr = p.split("POA")
#         print(curr)
        postcode.append(int(curr[1]))
    df['Postcode'] = postcode
    return df

In [50]:
get_postcode(monetary_data2011)
get_postcode(monetary_data2016)
get_postcode(monetary_data2021)

,Postcode,Median_age_persons_2021,Median_mortgage_repay_monthly_2021,Median_tot_prsnl_inc_weekly_2021,Median_rent_weekly_2021,Median_tot_fam_inc_weekly_2021,Average_num_psns_per_bedroom_2021,Average_household_size_2021,Suburb
0,3000,28,1733,768,370,1857,1.1,1.7,Melbourne
1,3002,42,2383,1532,480,3733,0.9,1.8,East Melbourne
2,3003,31,2106,1058,388,2406,1.1,1.9,West Melbourne
3,3004,37,2000,1343,451,2727,1.0,1.7,"St Kilda Road Melbourne, St Kilda Road Central..."
4,3006,31,1900,1171,411,2450,1.0,1.8,"South Wharf, Southbank"
...,...,...,...,...,...,...,...,...,...
689,3990,56,2134,621,308,1687,0.8,2.5,Glen Forbes
690,3991,49,1500,550,225,1674,0.7,2.4,Bass
691,3992,43,1517,699,280,1601,0.7,2.4,"Blackwood Forest, Dalyston, Ryanston, West Creek"
692,3995,50,1300,569,280,1392,0.7,2.2,"Kilcunda, North Wonthaggi, Cape Paterson, Ande..."


In [51]:
monetary_data2016

,Postcode,Median_age_persons_2016,Median_mortgage_repay_monthly_2016,Median_tot_prsnl_inc_weekly_2016,Median_rent_weekly_2016,Median_tot_fam_inc_weekly_2016,Average_num_psns_per_bedroom_2016,Average_household_size_2016,Suburb
0,3000,26,1700,431,451,1422,1.2,2.0,Melbourne
1,3002,38,2192,1341,451,3120,1.0,1.9,East Melbourne
2,3003,30,2006,852,450,2246,1.1,2.2,West Melbourne
3,3004,35,2000,1187,473,2526,1.0,1.8,"St Kilda Road Melbourne, St Kilda Road Central..."
4,3005,33,2184,1139,584,2562,1.0,2.1,World Trade Centre
...,...,...,...,...,...,...,...,...,...
693,3990,51,1467,641,275,1312,0.6,2.2,Glen Forbes
694,3991,50,1355,428,200,1009,0.7,2.4,Bass
695,3992,42,1419,543,225,1256,0.7,2.3,"Blackwood Forest, Dalyston, Ryanston, West Creek"
696,3995,48,1300,480,230,1129,0.7,2.2,"Kilcunda, North Wonthaggi, Cape Paterson, Ande..."


In [52]:
monetary_data2011

,Postcode,Median_age_persons_2011,Median_mortgage_repay_monthly_2011,Median_Tot_prsnl_inc_weekly_2011,Median_rent_weekly__2011,Median_Tot_fam_inc_weekly_2011,Average_num_psns_per_bedroom_2011,Average_household_size_2011,Suburb
0,3000,26,1950,545,402,1446,1.3,1.8,Melbourne
1,3002,36,2383,1164,415,2693,1.1,1.8,East Melbourne
2,3003,30,2400,800,441,2006,1.2,2.2,West Melbourne
3,3004,34,2167,1103,450,2544,1.2,1.9,"St Kilda Road Melbourne, St Kilda Road Central..."
4,3005,30,2167,1056,500,2416,1.2,2.0,World Trade Centre
...,...,...,...,...,...,...,...,...,...
662,3988,40,1343,521,165,1214,1.1,2.7,"Poowong North, Mountain View, Poowong, Poowong..."
663,3991,47,1083,461,160,1062,1.1,2.5,Bass
664,3992,39,1292,525,195,1174,1.1,2.4,"Blackwood Forest, Dalyston, Ryanston, West Creek"
665,3995,44,1300,445,220,1064,1.1,2.3,"Kilcunda, North Wonthaggi, Cape Paterson, Ande..."


########################################################################################

In [53]:
query = """SELECT * FROM monetary_data2011 INNER JOIN monetary_data2016 
on monetary_data2011.Postcode = monetary_data2016.Postcode"""
joined = pysqldf(query)

In [54]:
joined

,Postcode,Median_age_persons_2011,Median_mortgage_repay_monthly_2011,Median_Tot_prsnl_inc_weekly_2011,Median_rent_weekly__2011,Median_Tot_fam_inc_weekly_2011,Average_num_psns_per_bedroom_2011,Average_household_size_2011,Suburb,Postcode,Median_age_persons_2016,Median_mortgage_repay_monthly_2016,Median_tot_prsnl_inc_weekly_2016,Median_rent_weekly_2016,Median_tot_fam_inc_weekly_2016,Average_num_psns_per_bedroom_2016,Average_household_size_2016,Suburb
0,3000,26,1950,545,402,1446,1.3,1.8,Melbourne,3000,26,1700,431,451,1422,1.2,2.0,Melbourne
1,3002,36,2383,1164,415,2693,1.1,1.8,East Melbourne,3002,38,2192,1341,451,3120,1.0,1.9,East Melbourne
2,3003,30,2400,800,441,2006,1.2,2.2,West Melbourne,3003,30,2006,852,450,2246,1.1,2.2,West Melbourne
3,3004,34,2167,1103,450,2544,1.2,1.9,"St Kilda Road Melbourne, St Kilda Road Central...",3004,35,2000,1187,473,2526,1.0,1.8,"St Kilda Road Melbourne, St Kilda Road Central..."
4,3005,30,2167,1056,500,2416,1.2,2.0,World Trade Centre,3005,33,2184,1139,584,2562,1.0,2.1,World Trade Centre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,3988,40,1343,521,165,1214,1.1,2.7,"Poowong North, Mountain View, Poowong, Poowong...",3988,45,1300,593,200,1370,0.8,2.6,"Poowong North, Mountain View, Poowong, Poowong..."
663,3991,47,1083,461,160,1062,1.1,2.5,Bass,3991,50,1355,428,200,1009,0.7,2.4,Bass
664,3992,39,1292,525,195,1174,1.1,2.4,"Blackwood Forest, Dalyston, Ryanston, West Creek",3992,42,1419,543,225,1256,0.7,2.3,"Blackwood Forest, Dalyston, Ryanston, West Creek"
665,3995,44,1300,445,220,1064,1.1,2.3,"Kilcunda, North Wonthaggi, Cape Paterson, Ande...",3995,48,1300,480,230,1129,0.7,2.2,"Kilcunda, North Wonthaggi, Cape Paterson, Ande..."


In [55]:
query = """SELECT * FROM joined INNER JOIN monetary_data2021 
on joined.Postcode = monetary_data2021.Postcode"""
final_joined = pysqldf(query)

In [56]:
final_joined

,Postcode,Median_age_persons_2011,Median_mortgage_repay_monthly_2011,Median_Tot_prsnl_inc_weekly_2011,Median_rent_weekly__2011,Median_Tot_fam_inc_weekly_2011,Average_num_psns_per_bedroom_2011,Average_household_size_2011,Suburb,Median_age_persons_2016,...,Average_household_size_2016,Postcode,Median_age_persons_2021,Median_mortgage_repay_monthly_2021,Median_tot_prsnl_inc_weekly_2021,Median_rent_weekly_2021,Median_tot_fam_inc_weekly_2021,Average_num_psns_per_bedroom_2021,Average_household_size_2021,Suburb
0,3000,26,1950,545,402,1446,1.3,1.8,Melbourne,26,...,2.0,3000,28,1733,768,370,1857,1.1,1.7,Melbourne
1,3002,36,2383,1164,415,2693,1.1,1.8,East Melbourne,38,...,1.9,3002,42,2383,1532,480,3733,0.9,1.8,East Melbourne
2,3003,30,2400,800,441,2006,1.2,2.2,West Melbourne,30,...,2.2,3003,31,2106,1058,388,2406,1.1,1.9,West Melbourne
3,3004,34,2167,1103,450,2544,1.2,1.9,"St Kilda Road Melbourne, St Kilda Road Central...",35,...,1.8,3004,37,2000,1343,451,2727,1.0,1.7,"St Kilda Road Melbourne, St Kilda Road Central..."
4,3006,29,2300,988,496,2235,1.2,1.9,"South Wharf, Southbank",30,...,1.9,3006,31,1900,1171,411,2450,1.0,1.8,"South Wharf, Southbank"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,3988,40,1343,521,165,1214,1.1,2.7,"Poowong North, Mountain View, Poowong, Poowong...",45,...,2.6,3988,42,1713,727,250,1914,0.8,2.6,"Poowong North, Mountain View, Poowong, Poowong..."
658,3991,47,1083,461,160,1062,1.1,2.5,Bass,50,...,2.4,3991,49,1500,550,225,1674,0.7,2.4,Bass
659,3992,39,1292,525,195,1174,1.1,2.4,"Blackwood Forest, Dalyston, Ryanston, West Creek",42,...,2.3,3992,43,1517,699,280,1601,0.7,2.4,"Blackwood Forest, Dalyston, Ryanston, West Creek"
660,3995,44,1300,445,220,1064,1.1,2.3,"Kilcunda, North Wonthaggi, Cape Paterson, Ande...",48,...,2.2,3995,50,1300,569,280,1392,0.7,2.2,"Kilcunda, North Wonthaggi, Cape Paterson, Ande..."


In [81]:
final_joined.to_csv('../data/curated/cleaned_monetary_data.csv', index = False)

#######################################################################################################

In [58]:
cols_to_use = monetary_data2016.columns.difference(monetary_data2011.columns)
dfNew = pd.merge(monetary_data2011, monetary_data2016[cols_to_use], left_index=True, right_index=True, how='outer')
# all_monetary_data = pd.merge(dfNew, monetary_data2021, left_index=True, right_index=True, how='outer')
# all_monetary_data

In [59]:
monetary_data2021.pop('Suburb')
all_monetary_data = pd.merge(dfNew, monetary_data2021, left_index=True, right_index=True, how='outer')

In [60]:
all_monetary_data.pop('Postcode_x')

0      3000.0
1      3002.0
2      3003.0
3      3004.0
4      3005.0
        ...  
693       NaN
694       NaN
695       NaN
696       NaN
697       NaN
Name: Postcode_x, Length: 698, dtype: float64

In [61]:
all_monetary_data.rename(columns = {'Postcode_y': 'Postcode'}, inplace = True)

In [62]:
cleaned_monetary_data = all_monetary_data.dropna()

In [63]:
cleaned_monetary_data

,Median_age_persons_2011,Median_mortgage_repay_monthly_2011,Median_Tot_prsnl_inc_weekly_2011,Median_rent_weekly__2011,Median_Tot_fam_inc_weekly_2011,Average_num_psns_per_bedroom_2011,Average_household_size_2011,Suburb,Average_household_size_2016,Average_num_psns_per_bedroom_2016,...,Median_tot_fam_inc_weekly_2016,Median_tot_prsnl_inc_weekly_2016,Postcode,Median_age_persons_2021,Median_mortgage_repay_monthly_2021,Median_tot_prsnl_inc_weekly_2021,Median_rent_weekly_2021,Median_tot_fam_inc_weekly_2021,Average_num_psns_per_bedroom_2021,Average_household_size_2021
0,26.0,1950.0,545.0,402.0,1446.0,1.3,1.8,Melbourne,2.0,1.2,...,1422,431,3000.0,28.0,1733.0,768.0,370.0,1857.0,1.1,1.7
1,36.0,2383.0,1164.0,415.0,2693.0,1.1,1.8,East Melbourne,1.9,1.0,...,3120,1341,3002.0,42.0,2383.0,1532.0,480.0,3733.0,0.9,1.8
2,30.0,2400.0,800.0,441.0,2006.0,1.2,2.2,West Melbourne,2.2,1.1,...,2246,852,3003.0,31.0,2106.0,1058.0,388.0,2406.0,1.1,1.9
3,34.0,2167.0,1103.0,450.0,2544.0,1.2,1.9,"St Kilda Road Melbourne, St Kilda Road Central...",1.8,1.0,...,2526,1187,3004.0,37.0,2000.0,1343.0,451.0,2727.0,1.0,1.7
4,30.0,2167.0,1056.0,500.0,2416.0,1.2,2.0,World Trade Centre,2.1,1.0,...,2562,1139,3006.0,31.0,1900.0,1171.0,411.0,2450.0,1.0,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,40.0,1343.0,521.0,165.0,1214.0,1.1,2.7,"Poowong North, Mountain View, Poowong, Poowong...",2.3,0.7,...,1332,557,3945.0,50.0,1634.0,761.0,300.0,1834.0,0.8,2.5
663,47.0,1083.0,461.0,160.0,1062.0,1.1,2.5,Bass,2.0,0.6,...,1379,621,3946.0,51.0,1733.0,787.0,285.0,1797.0,0.7,2.4
664,39.0,1292.0,525.0,195.0,1174.0,1.1,2.4,"Blackwood Forest, Dalyston, Ryanston, West Creek",1.9,0.6,...,1579,678,3950.0,47.0,1338.0,619.0,270.0,1541.0,0.7,2.3
665,44.0,1300.0,445.0,220.0,1064.0,1.1,2.3,"Kilcunda, North Wonthaggi, Cape Paterson, Ande...",1.8,0.5,...,2263,1147,3951.0,49.0,1517.0,716.0,216.0,1783.0,0.8,2.6


In [236]:
cleaned_monetary_data.rename(columns = {"Median_Tot_prsnl_inc_weekly_2011": "Median_tot_prsnl_inc_weekly_2011",
                                       "Median_Tot_fam_inc_weekly_2011": "Median_tot_fam_inc_weekly_2011"}, inplace = True)

/tmp/ipykernel_74/3839122099.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_monetary_data.rename(columns = {"Median_Tot_prsnl_inc_weekly_2011": "Median_tot_prsnl_inc_weekly_2011",


In [237]:
cleaned_monetary_data

,Median_age_persons_2011,Median_mortgage_repay_monthly_2011,Median_tot_prsnl_inc_weekly_2011,Median_rent_weekly__2011,Median_tot_fam_inc_weekly_2011,Average_num_psns_per_bedroom_2011,Average_household_size_2011,Suburb,Average_household_size_2016,Average_num_psns_per_bedroom_2016,...,Median_tot_fam_inc_weekly_2016,Median_tot_prsnl_inc_weekly_2016,Postcode,Median_age_persons_2021,Median_mortgage_repay_monthly_2021,Median_tot_prsnl_inc_weekly_2021,Median_rent_weekly_2021,Median_tot_fam_inc_weekly_2021,Average_num_psns_per_bedroom_2021,Average_household_size_2021
0,26.0,1950.0,545.0,402.0,1446.0,1.3,1.8,Melbourne,2.0,1.2,...,1422,431,POA3000,28.0,1733.0,768.0,370.0,1857.0,1.1,1.7
1,36.0,2383.0,1164.0,415.0,2693.0,1.1,1.8,East Melbourne,1.9,1.0,...,3120,1341,POA3002,42.0,2383.0,1532.0,480.0,3733.0,0.9,1.8
2,30.0,2400.0,800.0,441.0,2006.0,1.2,2.2,West Melbourne,2.2,1.1,...,2246,852,POA3003,31.0,2106.0,1058.0,388.0,2406.0,1.1,1.9
3,34.0,2167.0,1103.0,450.0,2544.0,1.2,1.9,"St Kilda Road Melbourne, St Kilda Road Central...",1.8,1.0,...,2526,1187,POA3004,37.0,2000.0,1343.0,451.0,2727.0,1.0,1.7
4,30.0,2167.0,1056.0,500.0,2416.0,1.2,2.0,World Trade Centre,2.1,1.0,...,2562,1139,POA3006,31.0,1900.0,1171.0,411.0,2450.0,1.0,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,40.0,1343.0,521.0,165.0,1214.0,1.1,2.7,"Poowong North, Mountain View, Poowong, Poowong...",2.3,0.7,...,1332,557,POA3945,50.0,1634.0,761.0,300.0,1834.0,0.8,2.5
663,47.0,1083.0,461.0,160.0,1062.0,1.1,2.5,Bass,2.0,0.6,...,1379,621,POA3946,51.0,1733.0,787.0,285.0,1797.0,0.7,2.4
664,39.0,1292.0,525.0,195.0,1174.0,1.1,2.4,"Blackwood Forest, Dalyston, Ryanston, West Creek",1.9,0.6,...,1579,678,POA3950,47.0,1338.0,619.0,270.0,1541.0,0.7,2.3
665,44.0,1300.0,445.0,220.0,1064.0,1.1,2.3,"Kilcunda, North Wonthaggi, Cape Paterson, Ande...",1.8,0.5,...,2263,1147,POA3951,49.0,1517.0,716.0,216.0,1783.0,0.8,2.6


### Adjusting for Inflation

In [83]:
need_adjust = ['Median_mortgage_repay_monthly', 'Median_tot_prsnl_inc_weekly', 'Median_rent_weekly', 'Median_tot_fam_inc_weekly']

In [84]:
for e in need_adjust:
    for c in cleaned_monetary_data:
        if e in c:
            year = c.split('_')[-1]
            if (year == '2011'):
                cleaned_monetary_data[c] = cleaned_monetary_data[c]/CPI_2011 * 100
            elif (year == '2016'):
                cleaned_monetary_data[c] = cleaned_monetary_data[c]/CPI_2016 * 100
            else:
                cleaned_monetary_data[c] = cleaned_monetary_data[c]/CPI_2021 * 100

In [87]:
cleaned_monetary_data = pd.read_csv('../data/curated/cleaned_monetary_data.csv')

In [88]:
cleaned_monetary_data

,Postcode,Median_age_persons_2011,Median_mortgage_repay_monthly_2011,Median_tot_prsnl_inc_weekly_2011,Median_rent_weekly__2011,Median_tot_fam_inc_weekly_2011,Average_num_psns_per_bedroom_2011,Average_household_size_2011,Suburb,Median_age_persons_2016,...,Average_household_size_2016,Postcode.1,Median_age_persons_2021,Median_mortgage_repay_monthly_2021,Median_tot_prsnl_inc_weekly_2021,Median_rent_weekly_2021,Median_tot_fam_inc_weekly_2021,Average_num_psns_per_bedroom_2021,Average_household_size_2021,Suburb.1
0,3000,26,1950,545,402,1446,1.3,1.8,Melbourne,26,...,2.0,3000,28,1733,768,370,1857,1.1,1.7,Melbourne
1,3002,36,2383,1164,415,2693,1.1,1.8,East Melbourne,38,...,1.9,3002,42,2383,1532,480,3733,0.9,1.8,East Melbourne
2,3003,30,2400,800,441,2006,1.2,2.2,West Melbourne,30,...,2.2,3003,31,2106,1058,388,2406,1.1,1.9,West Melbourne
3,3004,34,2167,1103,450,2544,1.2,1.9,"St Kilda Road Melbourne, St Kilda Road Central...",35,...,1.8,3004,37,2000,1343,451,2727,1.0,1.7,"St Kilda Road Melbourne, St Kilda Road Central..."
4,3006,29,2300,988,496,2235,1.2,1.9,"South Wharf, Southbank",30,...,1.9,3006,31,1900,1171,411,2450,1.0,1.8,"South Wharf, Southbank"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,3988,40,1343,521,165,1214,1.1,2.7,"Poowong North, Mountain View, Poowong, Poowong...",45,...,2.6,3988,42,1713,727,250,1914,0.8,2.6,"Poowong North, Mountain View, Poowong, Poowong..."
658,3991,47,1083,461,160,1062,1.1,2.5,Bass,50,...,2.4,3991,49,1500,550,225,1674,0.7,2.4,Bass
659,3992,39,1292,525,195,1174,1.1,2.4,"Blackwood Forest, Dalyston, Ryanston, West Creek",42,...,2.3,3992,43,1517,699,280,1601,0.7,2.4,"Blackwood Forest, Dalyston, Ryanston, West Creek"
660,3995,44,1300,445,220,1064,1.1,2.3,"Kilcunda, North Wonthaggi, Cape Paterson, Ande...",48,...,2.2,3995,50,1300,569,280,1392,0.7,2.2,"Kilcunda, North Wonthaggi, Cape Paterson, Ande..."


In [79]:
cleaned_monetary_data.to_csv('../data/curated/cleaned_monetary_data.csv', index = False)

In [89]:
yrs = [2012,2013,2014,2015,2017,2018,2019,2020]
for f in col:
    for yr in yrs:
        cleaned_monetary_data[f+'_'+str(yr)] = -1
    
cleaned_monetary_data

,Postcode,Median_age_persons_2011,Median_mortgage_repay_monthly_2011,Median_tot_prsnl_inc_weekly_2011,Median_rent_weekly__2011,Median_tot_fam_inc_weekly_2011,Average_num_psns_per_bedroom_2011,Average_household_size_2011,Suburb,Median_age_persons_2016,...,Average_num_psns_per_bedroom_2019,Average_num_psns_per_bedroom_2020,Average_household_size_2012,Average_household_size_2013,Average_household_size_2014,Average_household_size_2015,Average_household_size_2017,Average_household_size_2018,Average_household_size_2019,Average_household_size_2020
0,3000,26,1950,545,402,1446,1.3,1.8,Melbourne,26,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,3002,36,2383,1164,415,2693,1.1,1.8,East Melbourne,38,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,3003,30,2400,800,441,2006,1.2,2.2,West Melbourne,30,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,3004,34,2167,1103,450,2544,1.2,1.9,"St Kilda Road Melbourne, St Kilda Road Central...",35,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,3006,29,2300,988,496,2235,1.2,1.9,"South Wharf, Southbank",30,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,3988,40,1343,521,165,1214,1.1,2.7,"Poowong North, Mountain View, Poowong, Poowong...",45,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
658,3991,47,1083,461,160,1062,1.1,2.5,Bass,50,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
659,3992,39,1292,525,195,1174,1.1,2.4,"Blackwood Forest, Dalyston, Ryanston, West Creek",42,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
660,3995,44,1300,445,220,1064,1.1,2.3,"Kilcunda, North Wonthaggi, Cape Paterson, Ande...",48,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [90]:
cleaned_monetary_data.to_csv('../data/curated/cleaned_monetary_data.csv', index = False)

In [73]:
col = ['Median_age_persons', 'Median_mortgage_repay_monthly', 'Median_tot_prsnl_inc_weekly', 'Median_rent_weekly', 'Median_tot_fam_inc_weekly',
      'Average_num_psns_per_bedroom', 'Average_household_size']


In [74]:
test = cleaned_monetary_data.iloc[0,:]

In [75]:
test

Median_age_persons_2011                      26.0
Median_mortgage_repay_monthly_2011    1964.240745
Median_Tot_prsnl_inc_weekly_2011            545.0
Median_rent_weekly__2011               404.935784
Median_Tot_fam_inc_weekly_2011             1446.0
                                         ...     
Average_household_size_2015                    -1
Average_household_size_2017                    -1
Average_household_size_2018                    -1
Average_household_size_2019                    -1
Average_household_size_2020                    -1
Name: 0, Length: 79, dtype: object

In [470]:
X_df = {}
for feature in col:
    f = []
    for t in test.iteritems():
        if feature in t[0]:
            f.append(t[1])
    X_df[feature] = f

In [471]:
X_df = pd.DataFrame.from_dict(X_df)

In [472]:
X_df

,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size
0,26.0,1964.240745,548.980106,404.935784,1456.560060,1.3,1.8
1,26.0,1558.917928,395.231545,413.571756,1303.988996,1.2,2.0
2,28.0,1451.119950,643.081432,309.817877,1554.950806,1.1,1.7
3,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0
4,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0
5,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0
6,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0
7,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0
8,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0
9,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0


In [473]:
X_df['year'] = [2011,2016,2021]+yrs

In [474]:
X_df

,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,year
0,26.0,1964.240745,548.980106,404.935784,1456.560060,1.3,1.8,2011
1,26.0,1558.917928,395.231545,413.571756,1303.988996,1.2,2.0,2016
2,28.0,1451.119950,643.081432,309.817877,1554.950806,1.1,1.7,2021
3,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2012
4,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2013
5,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2014
6,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2015
7,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2017
8,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2018
9,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2019


In [475]:
X_df.loc[:,X_df.columns != 'Median_age_persons']

,Median_age_persons,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,year
0,26.0,548.980106,404.935784,1456.560060,1.3,1.8,2011
1,26.0,395.231545,413.571756,1303.988996,1.2,2.0,2016
2,28.0,643.081432,309.817877,1554.950806,1.1,1.7,2021
3,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2012
4,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2013
5,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2014
6,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2015
7,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2017
8,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2018
9,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2019


In [476]:
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt

In [493]:
X_df['Median_age_persons'][0:3].values.reshape((-1,1))

array([[26.],
       [26.],
       [28.]])

In [494]:
X_df['year'][0:3].values.reshape((-1,1))

array([[2011],
       [2016],
       [2021]])

In [495]:
reg = LinearRegression().fit(X_df['year'][0:3].values.reshape((-1,1)), X_df['Median_age_persons'][0:3].values.reshape((-1,1)))

In [496]:
reg.coef_

array([[0.2]])

In [497]:
X_df

,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,year
0,26.0,1964.240745,548.980106,404.935784,1456.560060,1.3,1.8,2011
1,26.0,1558.917928,395.231545,413.571756,1303.988996,1.2,2.0,2016
2,28.0,1451.119950,643.081432,309.817877,1554.950806,1.1,1.7,2021
3,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2012
4,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2013
5,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2014
6,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2015
7,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2017
8,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2018
9,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2019


In [498]:
reg.predict(np.array([[2020]]))

array([[27.46666667]])

In [499]:
years = {2012:-1,2013:-1,2014:-1,2015:-1,2017:-1,2018:-1,2019:-1,2020:-1}
for yr in years.keys():
    years[yr] = reg.predict(np.array([[yr]]))[0][0]

In [500]:
years

{2012: 25.866666666666674,
 2013: 26.066666666666663,
 2014: 26.26666666666665,
 2015: 26.46666666666664,
 2017: 26.866666666666674,
 2018: 27.066666666666663,
 2019: 27.26666666666665,
 2020: 27.46666666666664}

In [485]:
new_df = []
for i in range(0, 11):
    row = X_df.iloc[i,:]
    for k in years.keys():
        if k == row['year']:
            row['Median_mortgage_repay_monthly'] = years[k]
         
    new_df.append(row)
    
    
new_df = pd.DataFrame(new_df)
new_df

/tmp/ipykernel_74/2513902747.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['Median_mortgage_repay_monthly'] = years[k]


,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,year
0,26.0,1964.240745,548.980106,404.935784,1456.560060,1.3,1.8,2011.0
1,26.0,1558.917928,395.231545,413.571756,1303.988996,1.2,2.0,2016.0
2,28.0,1451.119950,643.081432,309.817877,1554.950806,1.1,1.7,2021.0
3,-1.0,1863.341192,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2012.0
4,-1.0,1812.029113,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2013.0
5,-1.0,1760.717033,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2014.0
6,-1.0,1709.404954,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2015.0
7,-1.0,1606.780795,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2017.0
8,-1.0,1555.468715,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2018.0
9,-1.0,1504.156636,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2019.0


In [486]:
new_df

,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Average_household_size,year
0,26.0,1964.240745,548.980106,404.935784,1456.560060,1.3,1.8,2011.0
1,26.0,1558.917928,395.231545,413.571756,1303.988996,1.2,2.0,2016.0
2,28.0,1451.119950,643.081432,309.817877,1554.950806,1.1,1.7,2021.0
3,-1.0,1863.341192,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2012.0
4,-1.0,1812.029113,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2013.0
5,-1.0,1760.717033,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2014.0
6,-1.0,1709.404954,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2015.0
7,-1.0,1606.780795,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2017.0
8,-1.0,1555.468715,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2018.0
9,-1.0,1504.156636,-1.000000,-1.000000,-1.000000,-1.0,-1.0,2019.0


In [488]:
all_yrs = {2011:0,2012:3,2013:4,2014:5,2015:6,2016:1,2017:7,2018:8,2019:9,2020:10,2021:2}
new_row = []
field = {}
for c in col:
    for yr in all_yrs.keys():
        field[c+'_'+str(yr)] = new_df[c][all_yrs[yr]]

s = pd.Series(field)
new_row.append(s)
        
new_row

[Median_age_persons_2011        26.0
 Median_age_persons_2012        -1.0
 Median_age_persons_2013        -1.0
 Median_age_persons_2014        -1.0
 Median_age_persons_2015        -1.0
                                ... 
 Average_household_size_2017    -1.0
 Average_household_size_2018    -1.0
 Average_household_size_2019    -1.0
 Average_household_size_2020    -1.0
 Average_household_size_2021     1.7
 Length: 77, dtype: float64]

In [508]:
X_df.loc[0:3, feature].values.reshape((-1,1))

array([[ 1.8],
       [ 2. ],
       [ 1.7],
       [-1. ]])

In [506]:
cleaned_monetary_data.loc[:,'Suburb']

0                                              Melbourne
1                                         East Melbourne
2                                         West Melbourne
3      St Kilda Road Melbourne, St Kilda Road Central...
4                                     World Trade Centre
                             ...                        
662    Poowong North, Mountain View, Poowong, Poowong...
663                                                 Bass
664     Blackwood Forest, Dalyston, Ryanston, West Creek
665    Kilcunda, North Wonthaggi, Cape Paterson, Ande...
666                               Pound Creek, Inverloch
Name: Suburb, Length: 667, dtype: object

In [489]:
df = pd.DataFrame(new_row)

In [490]:
df

,Median_age_persons_2011,Median_age_persons_2012,Median_age_persons_2013,Median_age_persons_2014,Median_age_persons_2015,Median_age_persons_2016,Median_age_persons_2017,Median_age_persons_2018,Median_age_persons_2019,Median_age_persons_2020,...,Average_household_size_2012,Average_household_size_2013,Average_household_size_2014,Average_household_size_2015,Average_household_size_2016,Average_household_size_2017,Average_household_size_2018,Average_household_size_2019,Average_household_size_2020,Average_household_size_2021
0,26.0,-1.0,-1.0,-1.0,-1.0,26.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,1.7


In [491]:
df['Suburb'] = cleaned_monetary_data['Suburb']

In [458]:
df

,Median_age_persons_2011,Median_age_persons_2012,Median_age_persons_2013,Median_age_persons_2014,Median_age_persons_2015,Median_age_persons_2016,Median_age_persons_2017,Median_age_persons_2018,Median_age_persons_2019,Median_age_persons_2020,...,Average_household_size_2013,Average_household_size_2014,Average_household_size_2015,Average_household_size_2016,Average_household_size_2017,Average_household_size_2018,Average_household_size_2019,Average_household_size_2020,Average_household_size_2021,Suburb
0,26.0,-1.0,-1.0,-1.0,-1.0,26.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,1.7,Melbourne


In [492]:
df.to_csv('../data/curated/test2.csv')

In [685]:
new_poa = []
for p in cleaned_monetary_data['Postcode']:
    new_poa.append(p.split("POA")[1])

In [687]:
cleaned_monetary_data['Postcode'] = new_poa

/tmp/ipykernel_74/3966051497.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_monetary_data['Postcode'] = new_poa


In [689]:
cleaned_monetary_data.to_csv('../data/curated/cleaned_monetary_data.csv', index = False)

In [690]:
cleaned_monetary_data

,Median_age_persons_2011,Median_mortgage_repay_monthly_2011,Median_tot_prsnl_inc_weekly_2011,Median_rent_weekly__2011,Median_tot_fam_inc_weekly_2011,Average_num_psns_per_bedroom_2011,Average_household_size_2011,Suburb,Average_household_size_2016,Average_num_psns_per_bedroom_2016,...,Average_num_psns_per_bedroom_2019,Average_num_psns_per_bedroom_2020,Average_household_size_2012,Average_household_size_2013,Average_household_size_2014,Average_household_size_2015,Average_household_size_2017,Average_household_size_2018,Average_household_size_2019,Average_household_size_2020
0,26.0,1964.240745,548.980106,404.935784,1456.560060,1.3,1.8,Melbourne,2.0,1.2,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,36.0,2400.402921,1172.500630,418.030723,2712.666835,1.1,1.8,East Melbourne,1.9,1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,30.0,2417.527071,805.842357,444.220599,2020.649710,1.2,2.2,West Melbourne,2.2,1.1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,34.0,2182.825485,1111.055150,453.286326,2562.578696,1.2,1.9,"St Kilda Road Melbourne, St Kilda Road Central...",1.8,1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,30.0,2182.825485,1063.711911,503.651473,2433.643918,1.2,2.0,World Trade Centre,2.1,1.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,40.0,1352.807857,524.804835,166.204986,1222.865777,1.1,2.7,"Poowong North, Mountain View, Poowong, Poowong...",2.3,0.7,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
663,47.0,1090.909091,464.366658,161.168471,1069.755729,1.1,2.5,Bass,2.0,0.6,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
664,39.0,1301.435407,528.834047,196.424075,1182.573659,1.1,2.4,"Blackwood Forest, Dalyston, Ryanston, West Creek",1.9,0.6,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
665,44.0,1309.493830,448.249811,221.606648,1071.770335,1.1,2.3,"Kilcunda, North Wonthaggi, Cape Paterson, Ande...",1.8,0.5,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


### Joining

In [25]:
joined = pd.merge(prop, suburb_stat, how='left', on='listing_id')
# joined_with_school = pd.merge(prop, schools, how='right', on='listing_id')

In [26]:
joined

,listing_id,price,beds,baths,parking,title,address,propertyType,status,bond,suburb,medianRentPrice,population,avgAge,owners,renters,families,singles
0,14303265,330.0,1,1,0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0,Docklands,380.0,10962.0,20 to 39,0.356243,0.643757,0.393689,0.606311
1,16089992,430.0,2,1,1,$430 pw,"38/60-68 Gladesville Boulevard, Patterson Lake...",Apartment / Unit / Flat,new,1868.0,Patterson Lakes,430.0,7565.0,40 to 59,0.835075,0.164925,0.527570,0.472430
2,16089977,420.0,3,1,1,$420,"8 Stoneman Turn, Caroline Springs VIC 3023",House,new,1820.0,Caroline Springs,410.0,24200.0,20 to 39,0.766538,0.233462,0.571566,0.428434
3,16089972,640.0,1,1,1,$640 per week,"1510/1 Freshwater Place, Southbank VIC 3006",Apartment / Unit / Flat,new,2781.0,Southbank,360.0,18706.0,20 to 39,0.347128,0.652872,0.312419,0.687581
4,16089969,550.0,4,2,2,$550 PER WEEK,"6 Plenty Drive, Warragul VIC 3820",House,new,2390.0,Warragul,450.0,15757.0,60+,0.757297,0.242703,0.505706,0.494294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12432,15434092,400.0,4,2,2,$400 Per Week pw,"7 Ebony Way, Tarneit VIC 3029",House,live,1738.0,Tarneit,400.0,34556.0,20 to 39,0.699615,0.300385,0.611806,0.388194
12433,14409092,435.0,1,1,0,$435 per week,"17/416 St Kilda Rd, Melbourne VIC 3004",Apartment / Unit / Flat,live,2500.0,Melbourne,375.0,0.0,NaN,0.000000,0.000000,0.000000,0.000000
12434,15430756,350.0,3,2,4,$350 Per Week pw,"16 Goulburn Court, Werribee VIC 3030",House,live,1521.0,Werribee,340.0,40348.0,20 to 39,0.679556,0.320444,0.463697,0.536303
12435,15428527,600.0,4,2,1,$600,"18 Jessie Street, Preston VIC 3072",House,live,2600.0,Preston,600.0,32856.0,20 to 39,0.588015,0.411985,0.399137,0.600863


In [9]:
joined_with_school

,listing_id,price,beds,baths,parking,title,address_x,propertyType,status,bond,...,educationLevel,name,distance,state,postCode,year,gender,type,address_y,url
0,14303265,330.0,1.0,1.0,0.0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0,...,combined,Eltham College - King Street Campus,708.921822,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN
1,14303265,330.0,1.0,1.0,0.0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0,...,secondary,Hester Hornbrook Academy - City Campus,802.849893,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN
2,14303265,330.0,1.0,1.0,0.0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0,...,secondary,Ozford College - Ozford College Campus,899.683164,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN
3,14303265,330.0,1.0,1.0,0.0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0,...,secondary,Ozford College,900.734191,VIC,3000,11-12,CoEd,Private,"Melbourne, VIC 3000",http://www.ozford.edu.au
4,14303265,330.0,1.0,1.0,0.0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0,...,primary,Docklands Primary School,916.135369,VIC,3008,Prep-6,CoEd,Government,"Docklands, VIC 3008",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167377,15384018,750.0,2.0,1.0,0.0,$750/W Furnished,"2509/228 La Trobe Street, Melbourne VIC 3000",Apartment / Unit / Flat,live,3258.0,...,secondary,Simonds Catholic College,942.720484,VIC,3003,7-12,Boys,Catholic,"West Melbourne, VIC 3003",http://www.sccmelb.catholic.edu.au
167378,15384018,750.0,2.0,1.0,0.0,$750/W Furnished,"2509/228 La Trobe Street, Melbourne VIC 3000",Apartment / Unit / Flat,live,3258.0,...,secondary,St Joseph's Flexible Learning Centre,969.075211,VIC,3051,7-12,CoEd,Catholic,"North Melbourne, VIC 3051",http://www.youthplus.edu.au
167379,15384018,750.0,2.0,1.0,0.0,$750/W Furnished,"2509/228 La Trobe Street, Melbourne VIC 3000",Apartment / Unit / Flat,live,3258.0,...,combined,Eltham College - King Street Campus,1012.666859,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN
167380,15384018,750.0,2.0,1.0,0.0,$750/W Furnished,"2509/228 La Trobe Street, Melbourne VIC 3000",Apartment / Unit / Flat,live,3258.0,...,primary,Carlton Gardens Primary School,1058.486337,VIC,3053,Prep-6,CoEd,Government,"Carlton, VIC 3053",http://www.carltongardens.vic.edu.au


In [10]:
joined_with_school.to_csv('../data/curated/joined_with_school.csv')

In [2]:
joined_with_school = pd.read_csv('../data/curated/joined_with_school.csv')

In [3]:
joined_with_school

,Unnamed: 0,listing_id,price,beds,baths,parking,title,address_x,propertyType,status,...,educationLevel,name,distance,state,postCode,year,gender,type,address_y,url
0,0,14303265,330.0,1.0,1.0,0.0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,...,combined,Eltham College - King Street Campus,708.921822,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN
1,1,14303265,330.0,1.0,1.0,0.0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,...,secondary,Hester Hornbrook Academy - City Campus,802.849893,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN
2,2,14303265,330.0,1.0,1.0,0.0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,...,secondary,Ozford College - Ozford College Campus,899.683164,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN
3,3,14303265,330.0,1.0,1.0,0.0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,...,secondary,Ozford College,900.734191,VIC,3000,11-12,CoEd,Private,"Melbourne, VIC 3000",http://www.ozford.edu.au
4,4,14303265,330.0,1.0,1.0,0.0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,...,primary,Docklands Primary School,916.135369,VIC,3008,Prep-6,CoEd,Government,"Docklands, VIC 3008",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167377,167377,15384018,750.0,2.0,1.0,0.0,$750/W Furnished,"2509/228 La Trobe Street, Melbourne VIC 3000",Apartment / Unit / Flat,live,...,secondary,Simonds Catholic College,942.720484,VIC,3003,7-12,Boys,Catholic,"West Melbourne, VIC 3003",http://www.sccmelb.catholic.edu.au
167378,167378,15384018,750.0,2.0,1.0,0.0,$750/W Furnished,"2509/228 La Trobe Street, Melbourne VIC 3000",Apartment / Unit / Flat,live,...,secondary,St Joseph's Flexible Learning Centre,969.075211,VIC,3051,7-12,CoEd,Catholic,"North Melbourne, VIC 3051",http://www.youthplus.edu.au
167379,167379,15384018,750.0,2.0,1.0,0.0,$750/W Furnished,"2509/228 La Trobe Street, Melbourne VIC 3000",Apartment / Unit / Flat,live,...,combined,Eltham College - King Street Campus,1012.666859,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN
167380,167380,15384018,750.0,2.0,1.0,0.0,$750/W Furnished,"2509/228 La Trobe Street, Melbourne VIC 3000",Apartment / Unit / Flat,live,...,primary,Carlton Gardens Primary School,1058.486337,VIC,3053,Prep-6,CoEd,Government,"Carlton, VIC 3053",http://www.carltongardens.vic.edu.au


In [94]:
monetary_data = pd.read_csv('../data/curated/all_imputed_monetary_data.csv')
monetary_data

,Median_age_persons_2011,Median_age_persons_2012,Median_age_persons_2013,Median_age_persons_2014,Median_age_persons_2015,Median_age_persons_2016,Median_age_persons_2017,Median_age_persons_2018,Median_age_persons_2019,Median_age_persons_2020,...,Average_household_size_2014,Average_household_size_2015,Average_household_size_2016,Average_household_size_2017,Average_household_size_2018,Average_household_size_2019,Average_household_size_2020,Average_household_size_2021,Suburb,Postcode
0,26.0,25.866667,26.066667,26.266667,26.466667,26.0,26.866667,27.066667,27.266667,27.466667,...,1.853333,1.843333,2.0,1.823333,1.813333,1.803333,1.793333,1.7,Melbourne,3000
1,36.0,36.266667,36.866667,37.466667,38.066667,38.0,39.266667,39.866667,40.466667,41.066667,...,1.833333,1.833333,1.9,1.833333,1.833333,1.833333,1.833333,1.8,East Melbourne,3002
2,30.0,29.933333,30.033333,30.133333,30.233333,30.0,30.433333,30.533333,30.633333,30.733333,...,2.160000,2.130000,2.2,2.070000,2.040000,2.010000,1.980000,1.9,West Melbourne,3003
3,34.0,34.133333,34.433333,34.733333,35.033333,35.0,35.633333,35.933333,36.233333,36.533333,...,1.840000,1.820000,1.8,1.780000,1.760000,1.740000,1.720000,1.7,"St Kilda Road Melbourne, St Kilda Road Central...",3004
4,29.0,29.200000,29.400000,29.600000,29.800000,30.0,30.200000,30.400000,30.600000,30.800000,...,1.886667,1.876667,1.9,1.856667,1.846667,1.836667,1.826667,1.8,"South Wharf, Southbank",3006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,40.0,41.533333,41.733333,41.933333,42.133333,45.0,42.533333,42.733333,42.933333,43.133333,...,2.653333,2.643333,2.6,2.623333,2.613333,2.603333,2.593333,2.6,"Poowong North, Mountain View, Poowong, Poowong...",3988
658,47.0,47.866667,48.066667,48.266667,48.466667,50.0,48.866667,49.066667,49.266667,49.466667,...,2.453333,2.443333,2.4,2.423333,2.413333,2.403333,2.393333,2.4,Bass,3991
659,39.0,39.733333,40.133333,40.533333,40.933333,42.0,41.733333,42.133333,42.533333,42.933333,...,2.366667,2.366667,2.3,2.366667,2.366667,2.366667,2.366667,2.4,"Blackwood Forest, Dalyston, Ryanston, West Creek",3992
660,44.0,44.933333,45.533333,46.133333,46.733333,48.0,47.933333,48.533333,49.133333,49.733333,...,2.253333,2.243333,2.2,2.223333,2.213333,2.203333,2.193333,2.2,"Kilcunda, North Wonthaggi, Cape Paterson, Ande...",3995


In [34]:
bed_1_flat = pd.read_csv('../data/raw/Rental_Price_historical/1_bed_flat.csv')
bed_1_flat

,Unnamed: 0,Unnamed: 1,Mar-00,Unnamed: 3,Jun-00,Unnamed: 5,Sep-00,Unnamed: 7,Dec-00,Unnamed: 9,...,Mar-20,Unnamed: 163,Jun-20,Unnamed: 165,Sep-20,Unnamed: 167,Dec-20,Unnamed: 169,Mar-21,Unnamed: 171
0,NaN,NaN,Count,Median,Count,Median,Count,Median,Count,Median,...,Count,Median,Count,Median,Count,Median,Count,Median,Count,Median
1,Inner Melbourne,Albert Park-Middle Park-West St Kilda,352,$165,346,$165,376,$170,367,$175,...,230,$370,228,$355,231,$350,243,$350,263,$340
2,NaN,Armadale,210,$150,210,$150,211,$155,211,$160,...,205,$375,208,$368,170,$350,184,$350,206,$350
3,NaN,Carlton North,86,$150,77,$155,74,$150,65,$150,...,70,$363,67,$360,72,$350,64,$350,68,$338
4,NaN,Carlton-Parkville,297,$170,297,$170,310,$175,343,$180,...,"3,354",$360,"3,239",$356,"2,919",$350,"2,829",$340,"2,097",$300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,NaN,Wanagaratta,51,$85,45,$85,43,$85,46,$85,...,69,$195,67,$195,61,$190,56,$195,52,$195
156,NaN,Warragul,13,$80,11,$75,12,$90,10,$90,...,-,-,-,-,-,-,-,-,-,-
157,NaN,Warrnambool,113,$75,103,$75,107,$75,103,$80,...,61,$200,65,$220,63,$220,63,$230,55,$250
158,NaN,Wodonga,76,$85,72,$85,77,$85,83,$85,...,46,$180,57,$185,56,$185,56,$185,59,$200


In [114]:
bed_1_flat = pd.read_csv('../data/raw/Rental_Price_historical/1_bed_flat.csv')
bed_2_flat = pd.read_csv('../data/raw/Rental_Price_historical/2_bed_flat.csv')
bed_2_house = pd.read_csv('../data/raw/Rental_Price_historical/2_bed_house.csv')
bed_3_flat = pd.read_csv('../data/raw/Rental_Price_historical/3_bed_flat.csv')
bed_3_house = pd.read_csv('../data/raw/Rental_Price_historical/3_bed_house.csv')
bed_4_house = pd.read_csv('../data/raw/Rental_Price_historical/4_bed_house.csv')

with open('../data/curated/postcode_dict', 'rb') as f:
    postcode_dict = pickle.load(f)

In [115]:
def clean_price_data(df, postcode_dict):
    df.iat[0,1] = 'Suburb'
    
    for i, c in enumerate(df):
        if (i % 2 == 0 and i != 0 and i != 1):
            df.iat[0,i] = 'Count'+'_'+c.split("-")[0]+'20'+c.split("-")[1]
            df.iat[0,i+1] = 'Median'+'_'+c.split("-")[0]+'20'+c.split("-")[1]
    
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header #set the header row as the df header
    df_clean = df.drop(df.columns[[0]],axis = 1)
    df_clean = df_clean.reset_index()
#     df_clean.pop('level_0')
    df_clean.pop('index')
    df_clean = df_clean[df_clean.Suburb != "Group Total"]
    df_clean = df_clean.reset_index()
    df_clean.pop('index')

    poa = []
    not_found = []
    for i in range(0, len(df_clean['Suburb'])):
        s = df_clean.loc[i,'Suburb']
        found = False
        for k in postcode_dict.keys():
            subs = str(postcode_dict[k]).split(",")
            for e in subs:
                if (e.strip() in s):
                    poa.append(int(k))
                    found = True
                    break
            if found:
                break
        if not found:
            poa.append(0)
            not_found.append((i, s))
            
    poa[142] = 4806 # manually fixing postcode for wangaratta
    poa[96] = 3139
    df_clean['Postcode'] = poa
    
    return df_clean

In [116]:
bed_1_flat = clean_price_data(bed_1_flat, postcode_dict)
bed_2_flat = clean_price_data(bed_2_flat, postcode_dict)
bed_2_house = clean_price_data(bed_2_house, postcode_dict)
bed_3_flat = clean_price_data(bed_3_flat, postcode_dict)
bed_3_house = clean_price_data(bed_3_house, postcode_dict)
bed_4_house = clean_price_data(bed_4_house, postcode_dict)

In [130]:
bed_1_flat.pop('Suburb')
bed_2_flat.pop('Suburb')
bed_2_house.pop('Suburb')
bed_3_flat.pop('Suburb')
bed_3_house.pop('Suburb')
bed_4_house.pop('Suburb')

0      Albert Park-Middle Park-West St Kilda
1                                   Armadale
2                              Carlton North
3                          Carlton-Parkville
4                            CBD-St Kilda Rd
                       ...                  
141                                Traralgon
142                              Wanagaratta
143                                 Warragul
144                              Warrnambool
145                                  Wodonga
Name: Suburb, Length: 146, dtype: object

In [124]:
bed_1_flat

,Count_Mar2000,Median_Mar2000,Count_Jun2000,Median_Jun2000,Count_Sep2000,Median_Sep2000,Count_Dec2000,Median_Dec2000,Count_Mar2001,Median_Mar2001,...,Median_Mar2020,Count_Jun2020,Median_Jun2020,Count_Sep2020,Median_Sep2020,Count_Dec2020,Median_Dec2020,Count_Mar2021,Median_Mar2021,Postcode
0,352,$165,346,$165,376,$170,367,$175,392,$180,...,$370,228,$355,231,$350,243,$350,263,$340,3182
1,210,$150,210,$150,211,$155,211,$160,204,$160,...,$375,208,$368,170,$350,184,$350,206,$350,3143
2,86,$150,77,$155,74,$150,65,$150,72,$160,...,$363,67,$360,72,$350,64,$350,68,$338,3053
3,297,$170,297,$170,310,$175,343,$180,352,$190,...,$360,"3,239",$356,"2,919",$350,"2,829",$340,"2,097",$300,3052
4,735,$250,844,$250,912,$250,931,$250,"1,118",$255,...,$440,"5,708",$425,"5,624",$400,"6,437",$360,"6,961",$330,3182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,108,$75,115,$75,111,$75,108,$75,114,$75,...,$190,86,$190,78,$200,79,$200,75,$200,3844
142,51,$85,45,$85,43,$85,46,$85,43,$85,...,$195,67,$195,61,$190,56,$195,52,$195,4806
143,13,$80,11,$75,12,$90,10,$90,13,$80,...,-,-,-,-,-,-,-,-,-,3820
144,113,$75,103,$75,107,$75,103,$80,96,$80,...,$200,65,$220,63,$220,63,$230,55,$250,3280


In [131]:
bed_1_flat.to_csv('../data/curated/Rental_Price_historical/1_bed_flat_cleaned.csv', index = False)
bed_2_flat.to_csv('../data/curated/Rental_Price_historical/2_bed_flat_cleaned.csv', index = False)
bed_2_house.to_csv('../data/curated/Rental_Price_historical/2_bed_house_cleaned.csv', index = False)
bed_3_flat.to_csv('../data/curated/Rental_Price_historical/3_bed_flat_cleaned.csv', index = False)
bed_3_house.to_csv('../data/curated/Rental_Price_historical/3_bed_house_cleaned.csv', index = False)
bed_4_house.to_csv('../data/curated/Rental_Price_historical/4_bed_house_cleaned.csv', index = False)

In [132]:
bed_1_flat_clean = pd.read_csv('../data/curated/Rental_Price_historical/1_bed_flat_cleaned.csv')
bed_1_flat_clean

,Count_Mar2000,Median_Mar2000,Count_Jun2000,Median_Jun2000,Count_Sep2000,Median_Sep2000,Count_Dec2000,Median_Dec2000,Count_Mar2001,Median_Mar2001,...,Median_Mar2020,Count_Jun2020,Median_Jun2020,Count_Sep2020,Median_Sep2020,Count_Dec2020,Median_Dec2020,Count_Mar2021,Median_Mar2021,Postcode
0,352,$165,346,$165,376,$170,367,$175,392,$180,...,$370,228,$355,231,$350,243,$350,263,$340,3182
1,210,$150,210,$150,211,$155,211,$160,204,$160,...,$375,208,$368,170,$350,184,$350,206,$350,3143
2,86,$150,77,$155,74,$150,65,$150,72,$160,...,$363,67,$360,72,$350,64,$350,68,$338,3053
3,297,$170,297,$170,310,$175,343,$180,352,$190,...,$360,"3,239",$356,"2,919",$350,"2,829",$340,"2,097",$300,3052
4,735,$250,844,$250,912,$250,931,$250,"1,118",$255,...,$440,"5,708",$425,"5,624",$400,"6,437",$360,"6,961",$330,3182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,108,$75,115,$75,111,$75,108,$75,114,$75,...,$190,86,$190,78,$200,79,$200,75,$200,3844
142,51,$85,45,$85,43,$85,46,$85,43,$85,...,$195,67,$195,61,$190,56,$195,52,$195,4806
143,13,$80,11,$75,12,$90,10,$90,13,$80,...,-,-,-,-,-,-,-,-,-,3820
144,113,$75,103,$75,107,$75,103,$80,96,$80,...,$200,65,$220,63,$220,63,$230,55,$250,3280


### Joining pricing datra with external datasets

In [141]:
monetary_data.nunique()

Median_age_persons_2011         41
Median_age_persons_2012        387
Median_age_persons_2013        413
Median_age_persons_2014        395
Median_age_persons_2015        414
                              ... 
Average_household_size_2019    198
Average_household_size_2020    203
Average_household_size_2021     20
Suburb                         662
Postcode                       662
Length: 79, dtype: int64

In [142]:
monetary_data

,Median_age_persons_2011,Median_age_persons_2012,Median_age_persons_2013,Median_age_persons_2014,Median_age_persons_2015,Median_age_persons_2016,Median_age_persons_2017,Median_age_persons_2018,Median_age_persons_2019,Median_age_persons_2020,...,Average_household_size_2014,Average_household_size_2015,Average_household_size_2016,Average_household_size_2017,Average_household_size_2018,Average_household_size_2019,Average_household_size_2020,Average_household_size_2021,Suburb,Postcode
0,26.0,25.866667,26.066667,26.266667,26.466667,26.0,26.866667,27.066667,27.266667,27.466667,...,1.853333,1.843333,2.0,1.823333,1.813333,1.803333,1.793333,1.7,Melbourne,3000
1,36.0,36.266667,36.866667,37.466667,38.066667,38.0,39.266667,39.866667,40.466667,41.066667,...,1.833333,1.833333,1.9,1.833333,1.833333,1.833333,1.833333,1.8,East Melbourne,3002
2,30.0,29.933333,30.033333,30.133333,30.233333,30.0,30.433333,30.533333,30.633333,30.733333,...,2.160000,2.130000,2.2,2.070000,2.040000,2.010000,1.980000,1.9,West Melbourne,3003
3,34.0,34.133333,34.433333,34.733333,35.033333,35.0,35.633333,35.933333,36.233333,36.533333,...,1.840000,1.820000,1.8,1.780000,1.760000,1.740000,1.720000,1.7,"St Kilda Road Melbourne, St Kilda Road Central...",3004
4,29.0,29.200000,29.400000,29.600000,29.800000,30.0,30.200000,30.400000,30.600000,30.800000,...,1.886667,1.876667,1.9,1.856667,1.846667,1.836667,1.826667,1.8,"South Wharf, Southbank",3006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,40.0,41.533333,41.733333,41.933333,42.133333,45.0,42.533333,42.733333,42.933333,43.133333,...,2.653333,2.643333,2.6,2.623333,2.613333,2.603333,2.593333,2.6,"Poowong North, Mountain View, Poowong, Poowong...",3988
658,47.0,47.866667,48.066667,48.266667,48.466667,50.0,48.866667,49.066667,49.266667,49.466667,...,2.453333,2.443333,2.4,2.423333,2.413333,2.403333,2.393333,2.4,Bass,3991
659,39.0,39.733333,40.133333,40.533333,40.933333,42.0,41.733333,42.133333,42.533333,42.933333,...,2.366667,2.366667,2.3,2.366667,2.366667,2.366667,2.366667,2.4,"Blackwood Forest, Dalyston, Ryanston, West Creek",3992
660,44.0,44.933333,45.533333,46.133333,46.733333,48.0,47.933333,48.533333,49.133333,49.733333,...,2.253333,2.243333,2.2,2.223333,2.213333,2.203333,2.193333,2.2,"Kilcunda, North Wonthaggi, Cape Paterson, Ande...",3995


In [150]:
bed_1_flat_clean

,Count_Mar2000,Median_Mar2000,Count_Jun2000,Median_Jun2000,Count_Sep2000,Median_Sep2000,Count_Dec2000,Median_Dec2000,Count_Mar2001,Median_Mar2001,...,Median_Mar2020,Count_Jun2020,Median_Jun2020,Count_Sep2020,Median_Sep2020,Count_Dec2020,Median_Dec2020,Count_Mar2021,Median_Mar2021,Postcode
0,352,$165,346,$165,376,$170,367,$175,392,$180,...,$370,228,$355,231,$350,243,$350,263,$340,3182
1,210,$150,210,$150,211,$155,211,$160,204,$160,...,$375,208,$368,170,$350,184,$350,206,$350,3143
2,86,$150,77,$155,74,$150,65,$150,72,$160,...,$363,67,$360,72,$350,64,$350,68,$338,3053
3,297,$170,297,$170,310,$175,343,$180,352,$190,...,$360,"3,239",$356,"2,919",$350,"2,829",$340,"2,097",$300,3052
4,735,$250,844,$250,912,$250,931,$250,"1,118",$255,...,$440,"5,708",$425,"5,624",$400,"6,437",$360,"6,961",$330,3182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,108,$75,115,$75,111,$75,108,$75,114,$75,...,$190,86,$190,78,$200,79,$200,75,$200,3844
142,51,$85,45,$85,43,$85,46,$85,43,$85,...,$195,67,$195,61,$190,56,$195,52,$195,4806
143,13,$80,11,$75,12,$90,10,$90,13,$80,...,-,-,-,-,-,-,-,-,-,3820
144,113,$75,103,$75,107,$75,103,$80,96,$80,...,$200,65,$220,63,$220,63,$230,55,$250,3280


In [147]:
# 1 bed flat
query = 'SELECT * FROM bed_1_flat_clean INNER JOIN monetary_data on bed_1_flat_clean.Postcode = monetary_data.Postcode'
joined = pysqldf(query)

query = 'SELECT * FROM bed_2_flat INNER JOIN monetary_data on bed_2_flat.Postcode = monetary_data.Postcode'
joined_bed_2_flat = pysqldf(query)

query = 'SELECT * FROM bed_2_house INNER JOIN monetary_data on bed_2_house.Postcode = monetary_data.Postcode'
joined_bed_2_house = pysqldf(query)

query = 'SELECT * FROM bed_3_flat INNER JOIN monetary_data on bed_3_flat.Postcode = monetary_data.Postcode'
joined_bed_3_flat = pysqldf(query)

query = 'SELECT * FROM bed_3_house INNER JOIN monetary_data on bed_3_house.Postcode = monetary_data.Postcode'
joined_bed_3_house = pysqldf(query)

query = 'SELECT * FROM bed_4_house INNER JOIN monetary_data on bed_4_house.Postcode = monetary_data.Postcode'
joined_bed_4_house = pysqldf(query)

In [148]:
joined

,Count_Mar2000,Median_Mar2000,Count_Jun2000,Median_Jun2000,Count_Sep2000,Median_Sep2000,Count_Dec2000,Median_Dec2000,Count_Mar2001,Median_Mar2001,...,Average_household_size_2014,Average_household_size_2015,Average_household_size_2016,Average_household_size_2017,Average_household_size_2018,Average_household_size_2019,Average_household_size_2020,Average_household_size_2021,Suburb,Postcode
0,352,$165,346,$165,376,$170,367,$175,392,$180,...,1.786667,1.776667,1.8,1.756667,1.746667,1.736667,1.726667,1.7,"St Kilda, St Kilda South, St Kilda West",3182
1,210,$150,210,$150,211,$155,211,$160,204,$160,...,2.086667,2.076667,2.1,2.056667,2.046667,2.036667,2.026667,2.0,"Armadale North, Armadale",3143
2,86,$150,77,$155,74,$150,65,$150,72,$160,...,1.886667,1.876667,1.9,1.856667,1.846667,1.836667,1.826667,1.8,"Carlton South, Carlton",3053
3,297,$170,297,$170,310,$175,343,$180,352,$190,...,2.240000,2.220000,2.2,2.180000,2.160000,2.140000,2.120000,2.1,"Melbourne University, Parkville",3052
4,735,$250,844,$250,912,$250,931,$250,"1,118",$255,...,1.786667,1.776667,1.8,1.756667,1.746667,1.736667,1.726667,1.7,"St Kilda, St Kilda South, St Kilda West",3182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,18,$91,20,$91,18,$90,18,$90,21,$95,...,2.700000,2.700000,2.7,2.700000,2.700000,2.700000,2.700000,2.7,"Torquay, Jan Juc, Bells Beach, Bellbrae",3228
141,108,$75,115,$75,111,$75,108,$75,114,$75,...,2.453333,2.443333,2.4,2.423333,2.413333,2.403333,2.393333,2.4,"Callignee, Willung South, Traralgon South, Car...",3844
142,13,$80,11,$75,12,$90,10,$90,13,$80,...,2.466667,2.466667,2.4,2.466667,2.466667,2.466667,2.466667,2.5,"Warragul, Bona Vista, Lillico",3820
143,113,$75,103,$75,107,$75,103,$80,96,$80,...,2.386667,2.376667,2.4,2.356667,2.346667,2.336667,2.326667,2.3,"Dennington, Warrnambool",3280


In [137]:
joined_bed_4_house

,Count_Mar2000,Median_Mar2000,Count_Jun2000,Median_Jun2000,Count_Sep2000,Median_Sep2000,Count_Dec2000,Median_Dec2000,Count_Mar2001,Median_Mar2001,...,Average_household_size_2014,Average_household_size_2015,Average_household_size_2016,Average_household_size_2017,Average_household_size_2018,Average_household_size_2019,Average_household_size_2020,Average_household_size_2021,Suburb,Postcode
0,19,$500,21,$550,24,$525,22,$513,24,$550,...,1.786667,1.776667,1.8,1.756667,1.746667,1.736667,1.726667,1.7,"St Kilda, St Kilda South, St Kilda West",3182
1,10,$615,11,$695,13,$850,13,$695,15,$760,...,2.086667,2.076667,2.1,2.056667,2.046667,2.036667,2.026667,2.0,"Armadale North, Armadale",3143
2,59,$380,58,$380,56,$383,53,$385,48,$403,...,1.886667,1.876667,1.9,1.856667,1.846667,1.836667,1.826667,1.8,"Carlton South, Carlton",3053
3,25,$450,25,$430,39,$450,42,$425,44,$420,...,2.240000,2.220000,2.2,2.180000,2.160000,2.140000,2.120000,2.1,"Melbourne University, Parkville",3052
4,-,-,-,-,-,-,-,-,-,-,...,1.786667,1.776667,1.8,1.756667,1.746667,1.736667,1.726667,1.7,"St Kilda, St Kilda South, St Kilda West",3182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,36,$198,38,$200,37,$200,32,$215,37,$230,...,2.700000,2.700000,2.7,2.700000,2.700000,2.700000,2.700000,2.7,"Torquay, Jan Juc, Bells Beach, Bellbrae",3228
141,64,$183,58,$180,52,$175,47,$175,48,$170,...,2.453333,2.443333,2.4,2.423333,2.413333,2.403333,2.393333,2.4,"Callignee, Willung South, Traralgon South, Car...",3844
142,38,$178,38,$180,37,$180,35,$185,33,$185,...,2.466667,2.466667,2.4,2.466667,2.466667,2.466667,2.466667,2.5,"Warragul, Bona Vista, Lillico",3820
143,85,$170,87,$180,82,$185,74,$188,76,$200,...,2.386667,2.376667,2.4,2.356667,2.346667,2.336667,2.326667,2.3,"Dennington, Warrnambool",3280


In [138]:
joined.to_csv('../data/curated/Rental_Price_historical/joined_1_bed_flat_monetary.csv', index=False)
joined_bed_2_flat.to_csv('../data/curated/Rental_Price_historical/joined_bed_2_flat_monetary.csv', index=False)
joined_bed_2_house.to_csv('../data/curated/Rental_Price_historical/joined_bed_2_house_monetary.csv', index=False)
joined_bed_3_flat.to_csv('../data/curated/Rental_Price_historical/joined_bed_3_flat_monetary.csv', index=False)
joined_bed_3_house.to_csv('../data/curated/Rental_Price_historical/joined_bed_3_house_monetary.csv', index=False)
joined_bed_4_house.to_csv('../data/curated/Rental_Price_historical/joined_bed_4_house_monetary.csv', index=False)

### Population data

In [162]:
pop = pd.read_csv('../data/raw/Population_by_suburb.csv')

In [163]:
pop

,Rank,Suburb,2011,2021,2031,Change_2011_2031
0,1,Tarneit,"21,566","44,509","74,932","53,366"
1,2,Wollert,871,"17,960","50,685","49,814"
2,3,Rockbank,"1,419","7,330","45,305","43,886"
3,4,Melton South,"9,054","26,694","47,002","37,948"
4,5,Officer,"1,894","15,391","39,495","37,601"
...,...,...,...,...,...,...
477,478,Meadow Heights,"14,955","14,470","14,587",-368
478,479,Kings Park,"8,575","8,318","8,137",-438
479,480,Gladstone Park,"8,939","8,563","8,449",-490
480,481,Aspendale Gardens,"6,957","6,564","6,399",-558


In [164]:
pop['2011'] = pd.to_numeric(pop['2011'].str.replace(',',''))
pop['2021'] = pd.to_numeric(pop['2021'].str.replace(',',''))
pop['2031'] = pd.to_numeric(pop['2031'].str.replace(',',''))
pop['Change_2011_2031'] = pd.to_numeric(pop['Change_2011_2031'].str.replace(',',''))

In [165]:
pop

,Rank,Suburb,2011,2021,2031,Change_2011_2031
0,1,Tarneit,21566,44509,74932,53366
1,2,Wollert,871,17960,50685,49814
2,3,Rockbank,1419,7330,45305,43886
3,4,Melton South,9054,26694,47002,37948
4,5,Officer,1894,15391,39495,37601
...,...,...,...,...,...,...
477,478,Meadow Heights,14955,14470,14587,-368
478,479,Kings Park,8575,8318,8137,-438
479,480,Gladstone Park,8939,8563,8449,-490
480,481,Aspendale Gardens,6957,6564,6399,-558


In [166]:
yrs = np.arange(2012, 2031, 1)
COLS = ["Pop", "year"]
yrs = np.delete(yrs, 9)


In [167]:
yrs = yrs.tolist()


In [168]:
np.arange(2011,2032).tolist()

[2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023,
 2024,
 2025,
 2026,
 2027,
 2028,
 2029,
 2030,
 2031]

In [178]:
df = {}
final = []
for i, row in pop.iterrows():
    row_dict = {}
    row_dict["year"] = [2011,2021,2031]
    row_dict['Pop'] = [row['2011'],row['2021'],row['2031']]
    row_df = pd.DataFrame(row_dict)
    reg = LinearRegression().fit(row_df[["year"]], row_df["Pop"])
    
    pred = []
    for yr in yrs:
        pred.extend(reg.predict(np.array([[yr]])))
    
    
    pop_dict = {}
    pop_dict["Pop"] = [row['2011']]+pred[0:9]+[row['2021']]+pred[9:]+[row['2031']]
    pop_dict['year'] = np.arange(2011,2032).tolist()
#     print(pop_dict)

#     print(pop_dict)
    curr = pd.DataFrame(pop_dict).T
    for col in curr:
        curr[col] = curr[col].astype(int)

    curr.columns = curr.iloc[1]
    curr = curr.drop('year')
    curr['Suburb'] = row['Suburb']
#     curr.reset_index()
#     print(curr)
#     if i == 0:
#         final = curr.copy()
#     else:
#         final.append(curr)
    final.append(curr.iloc[0,:])

In [179]:
final = pd.DataFrame(final)

In [182]:
final.to_csv('../data/curated/Pop_imputed_2011-2031.csv', index = False)

In [180]:
final

year,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,...,2023,2024,2025,2026,2027,2028,2029,2030,2031,Suburb
Pop,21566,22987,25655,28324,30992,33660,36329,38997,41665,44334,...,52338,55007,57675,60343,63012,65680,68348,71017,74932,Tarneit
Pop,871,755,3246,5737,8227,10718,13209,15699,18190,20681,...,28153,30644,33134,35625,38116,40606,43097,45588,50685,Wollert
Pop,1419,-1730,463,2657,4852,7046,9240,11435,13629,15823,...,22406,24600,26795,28989,31183,33378,35572,37766,45305,Rockbank
Pop,9054,10506,12404,14301,16198,18096,19993,21891,23788,25685,...,31378,33275,35172,37070,38967,40865,42762,44659,47002,Melton South
Pop,1894,2006,3886,5766,7646,9526,11406,13286,15166,17046,...,22686,24566,26446,28326,30206,32087,33967,35847,39495,Officer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pop,14955,14836,14817,14799,14781,14762,14744,14725,14707,14689,...,14633,14615,14597,14578,14560,14541,14523,14505,14587,Meadow Heights
Pop,8575,8540,8518,8496,8474,8452,8430,8409,8387,8365,...,8299,8277,8255,8233,8211,8190,8168,8146,8137,Kings Park
Pop,8939,8870,8846,8821,8797,8772,8748,8723,8699,8674,...,8601,8576,8552,8527,8503,8478,8454,8429,8449,Gladstone Park
Pop,6957,6891,6863,6835,6807,6779,6751,6723,6695,6667,...,6584,6556,6528,6500,6472,6444,6416,6388,6399,Aspendale Gardens


In [184]:
pop_df = pd.read_csv('../data/curated/Pop_imputed_2011-2031.csv')
pop_df

,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,...,2023,2024,2025,2026,2027,2028,2029,2030,2031,Suburb
0,21566,22987,25655,28324,30992,33660,36329,38997,41665,44334,...,52338,55007,57675,60343,63012,65680,68348,71017,74932,Tarneit
1,871,755,3246,5737,8227,10718,13209,15699,18190,20681,...,28153,30644,33134,35625,38116,40606,43097,45588,50685,Wollert
2,1419,-1730,463,2657,4852,7046,9240,11435,13629,15823,...,22406,24600,26795,28989,31183,33378,35572,37766,45305,Rockbank
3,9054,10506,12404,14301,16198,18096,19993,21891,23788,25685,...,31378,33275,35172,37070,38967,40865,42762,44659,47002,Melton South
4,1894,2006,3886,5766,7646,9526,11406,13286,15166,17046,...,22686,24566,26446,28326,30206,32087,33967,35847,39495,Officer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,14955,14836,14817,14799,14781,14762,14744,14725,14707,14689,...,14633,14615,14597,14578,14560,14541,14523,14505,14587,Meadow Heights
478,8575,8540,8518,8496,8474,8452,8430,8409,8387,8365,...,8299,8277,8255,8233,8211,8190,8168,8146,8137,Kings Park
479,8939,8870,8846,8821,8797,8772,8748,8723,8699,8674,...,8601,8576,8552,8527,8503,8478,8454,8429,8449,Gladstone Park
480,6957,6891,6863,6835,6807,6779,6751,6723,6695,6667,...,6584,6556,6528,6500,6472,6444,6416,6388,6399,Aspendale Gardens


### Crime Rates